In [7]:
float('inf')

inf

In [2]:
import json
import random
import csv

In [3]:
with open('./flask_backend/fab_oht_layout_2nd.json') as f:
    layout_data = json.load(f)
nodes = layout_data['nodes']

In [4]:
random.choice(nodes)

{'id': 'rn_InterBay3_10',
 'x': 198623.622047244,
 'y': 137500.0,
 'zcu_name': 'ZCU_A14_6',
 'zcu_type': 'RESET'}

In [5]:

# 테스트용 job_list 생성
def generate_random_job_list(nodes, num_ohts=600):
    oht_list = []
    available_nodes = nodes.copy()

    for _ in range(num_ohts):
        start_node = random.choice(nodes) 
        while start_node['id'] in oht_list:
            start_node = random.choice(nodes)        
    
        oht_list.append({
            'start_node': start_node['id'],
        })
    
    return oht_list

# job_list를 csv로 저장
def save_job_list_to_csv(job_list, filename='test_oht_list.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['start_node'])
        writer.writeheader()
        writer.writerows(job_list)

# job_list 생성
oht_list = generate_random_job_list(nodes, num_ohts=700)

# csv로 저장
save_job_list_to_csv(oht_list)

In [14]:
ports

[{'name': 'A10_LSTB_100_Port',
  'port_type': 'LSTB',
  'eqp_name': ',',
  'rail_line': 'rl_A10_18_19',
  'from_node': 'rn_A10_18',
  'to_node': 'rn_A10_19',
  'distance': 9167,
  'x': 144996.85043307,
  'y': 115624.666693239},
 {'name': 'A10_LSTB_101_Port',
  'port_type': 'LSTB',
  'eqp_name': ',',
  'rail_line': 'rl_A10_18_19',
  'from_node': 'rn_A10_18',
  'to_node': 'rn_A10_19',
  'distance': 9792,
  'x': 146996.85043307,
  'y': 114999.666693239},
 {'name': 'A10_LSTB_102_Port',
  'port_type': 'LSTB',
  'eqp_name': ',',
  'rail_line': 'rl_A10_20_21',
  'from_node': 'rn_A10_20',
  'to_node': 'rn_A10_21',
  'distance': 1042,
  'x': 144996.85043307,
  'y': 111249.666693239},
 {'name': 'A10_LSTB_103_Port',
  'port_type': 'LSTB',
  'eqp_name': ',',
  'rail_line': 'rl_A10_20_21',
  'from_node': 'rn_A10_20',
  'to_node': 'rn_A10_21',
  'distance': 1667,
  'x': 146996.85043307,
  'y': 110624.666693239},
 {'name': 'A10_LSTB_104_Port',
  'port_type': 'LSTB',
  'eqp_name': ',',
  'rail_line': 

In [1]:
import multiprocessing
import random
import time

# OHT 객체 클래스
class OHT:
    def __init__(self, id, position):
        self.id = id
        self.position = position

    def update_position(self, increment):
        self.position += increment

    def move(self, time_step):
        # 위치를 임의로 업데이트 (단순히 위치 + 속도)
        self.update_position(time_step)

    def get_position(self):
        return self.position

# 작업을 실행할 프로세스 함수
def process_oht(oht, time_step, lock):
    with lock:  # Lock을 사용하여 동기화
        oht.move(time_step)
        print(f"OHT {oht.id} new position: {oht.get_position()}")

# 병렬화 처리 함수
def parallel_oht_processing(oht_objects, time_step, process_count=4):
    # Lock을 사용하여 동기화 처리
    lock = multiprocessing.Lock()

    # multiprocessing.Pool로 프로세스 생성
    with multiprocessing.Pool(processes=process_count) as pool:
        pool.starmap(process_oht, [(oht, time_step, lock) for oht in oht_objects])

    # 최종 결과 출력
    for oht in oht_objects:
        print(f"OHT {oht.id} final position: {oht.get_position()}")

if __name__ == "__main__":
    # OHT 객체 생성
    oht_objects = [OHT(i, random.randint(0, 100)) for i in range(10)]

    # 병렬 처리 실행
    time_step = 1.0
    parallel_oht_processing(oht_objects, time_step, process_count=4)


RuntimeError: Lock objects should only be shared between processes through inheritance

In [3]:
import multiprocessing
import random

# OHT 객체 클래스
class OHT:
    def __init__(self, id, position):
        self.id = id
        self.position = position

    def update_position(self, increment):
        self.position += increment

    def get_position(self):
        return self.position

# 작업을 실행할 프로세스 함수
def process_oht(oht, time_step):
    oht.update_position(time_step)  # 객체의 값을 수정
    return oht  # 수정된 객체를 리턴

# 병렬화 처리 함수
def parallel_oht_processing(oht_objects, time_step, process_count=4):
    with multiprocessing.Manager() as manager:
        # 공유 객체 리스트
        shared_oht_objects = manager.list(oht_objects)
        
        # multiprocessing.Pool로 프로세스 생성
        with multiprocessing.Pool(processes=process_count) as pool:
            updated_ohts = pool.map(process_oht, [(oht, time_step) for oht in shared_oht_objects])

        # 결과 출력
        for oht in updated_ohts:
            print(f"OHT {oht.id} final position: {oht.get_position()}")

if __name__ == "__main__":
    # 멀티프로세싱 시작 방법 설정
    multiprocessing.set_start_method('spawn')  # spawn 방식으로 프로세스 시작

    # OHT 객체 생성
    oht_objects = [OHT(i, random.randint(0, 100)) for i in range(10)]

    # 병렬 처리 실행
    time_step = 1.0
    parallel_oht_processing(oht_objects, time_step, process_count=4)


RuntimeError: context has already been set

In [11]:
import multiprocessing
import time

class OHT:
    def __init__(self, id, position):
        self.id = id
        self.position = position

    def move(self):
        self.position += 1  # 간단히 이동시키기

    def get_position(self):
        return self.position

def process_oht(oht, queue):
    oht.move()  # OHT 이동
    print(oht)
    queue.put(oht)  # 결과를 Queue에 넣기

def update_oht_positions(oht_objects):
    
    
    # Queue 객체 생성
    queue = multiprocessing.Queue()
        # Pool을 사용하여 병렬 처리
    with multiprocessing.Pool(processes=4) as pool:
        for oht in oht_objects:
            pool.apply_async(process_oht, args=(oht, queue))  # OHT 처리
        pool.close()
        pool.join()
    
    # Queue에서 결과를 가져와 리스트로 반환
    updated_ohts = []
    while not queue.empty():
        updated_ohts.append(queue.get())
    return updated_ohts

if __name__ == '__main__':
    # OHT 객체 리스트 생성
    oht_objects = [OHT(i, i * 10) for i in range(10)]

    # OHT 객체 이동 업데이트
    updated_ohts = update_oht_positions(oht_objects)
    
    # 결과 출력
    for oht in updated_ohts:
        print(f"OHT {oht.id} final position: {oht.get_position()}")


In [1]:
import ray

ray.init()

class MyClass:
    def __init__(self, name):
        self.name = name

    @ray.remote
    def say_hello(self):
        return f"Hello from {self.name}"

# 객체 생성
obj = MyClass("Ray")

# 병렬 실행
futures = [obj.say_hello.remote(obj) for _ in range(10)]
results = ray.get(futures)

print(results)

ray.shutdown()


2025-01-22 08:27:02,942	INFO worker.py:1821 -- Started a local Ray instance.


['Hello from Ray', 'Hello from Ray', 'Hello from Ray', 'Hello from Ray', 'Hello from Ray', 'Hello from Ray', 'Hello from Ray', 'Hello from Ray', 'Hello from Ray', 'Hello from Ray']


In [2]:
import ray

ray.init()

class MyClass:
    def __init__(self, name):
        self.name = name

    @ray.remote
    def say_hello(self):
        return f"Hello from {self.name}"

# 객체 생성
obj = MyClass("Ray")

# 병렬 실행 시, 객체를 전달
futures = [obj.say_hello.remote() for _ in range(10)]
results = ray.get(futures)

print(results)

ray.shutdown()


2025-01-22 08:29:35,041	INFO worker.py:1821 -- Started a local Ray instance.


TypeError: missing a required argument: 'self'

In [1]:
import ray

# 1. 클래스를 ray.actor로 병렬화
@ray.remote
class MyClass:
    def __init__(self, value):
        self.value = value

    def increment(self, x):
        self.value += x
        return self.value

    def get_value(self):
        return self.value

# 2. `actor` 인스턴스 생성
ray.init(ignore_reinit_error=True)  # Ray 클러스터 초기화
my_class_instance = MyClass.remote(10)  # `MyClass` 인스턴스를 Ray에서 실행

# 3. 메서드를 병렬로 실행
future1 = my_class_instance.increment.remote(5)  # `increment` 메서드를 병렬로 실행
future2 = my_class_instance.increment.remote(10)  # `increment` 메서드를 병렬로 실행

# 4. 결과를 가져옴
result1 = ray.get(future1)  # 병렬 실행된 결과 가져오기
result2 = ray.get(future2)

print("Result 1:", result1)  # 15
print("Result 2:", result2)  # 20

# 5. 클래스 상태 확인
value = ray.get(my_class_instance.get_value.remote())  # 클래스의 값을 확인
print("Final value:", value)  # 20

# Ray 클러스터 종료
ray.shutdown()


2025-01-23 01:45:00,856	INFO worker.py:1821 -- Started a local Ray instance.


Result 1: 15
Result 2: 25
Final value: 25


In [3]:
@ray.remote
class Counter:
    def __init__(self, start=0):
        self.value = start

    def increment(self, x):
        self.value += x
        return self.value

    def get_value(self):
        return self.value

# `Counter` 클래스를 병렬로 실행하는 예시
counter = Counter.remote(0)  # 초기 값 0으로 `Counter` 인스턴스 생성

# 병렬로 실행된 메서드
futures = [counter.increment.remote(i) for i in range(5)]
results = ray.get(futures)

print("Increment results:", results)  # [0, 1, 3, 6, 10]

# 최종 값 확인
final_value = ray.get(counter.get_value.remote())
print("Final value:", final_value)  # 10

# Ray 클러스터 종료
ray.shutdown()


2025-01-23 01:47:47,639	INFO worker.py:1821 -- Started a local Ray instance.


Increment results: [0, 1, 3, 6, 10]
Final value: 10


In [ ]:

#import libraries
import numpy as np
import random
import networkit as nk
import json
import threading
import ray

def compress_data(data):
    json_data = json.dumps(data).encode('utf-8')
    compressed_data = gzip.compress(json_data)
    return base64.b64encode(compressed_data).decode('utf-8')

#node 클래스
class node():
    def __init__(self, id, coord):
        #node id
        self.id = id
        #node 좌표 (2차원)
        self.coord = np.array(coord)
        #노드로 들어오는 edge들 => intersection 쪽 충돌 감지 위해 필요
        self.incoming_edges = []
        self.outgoing_edges = []
        self.OHT = None
        
class edge():
    def __init__(self, source, dest, length, max_speed):
        self.id = None
        #Edge source
        self.source = source
        source.outgoing_edges.append(self)
        #edge target(destination)
        self.dest = dest
        dest.incoming_edges.append(self)
        #edge length
        self.length = length
        #edge의 방향 벡터
        self.unit_vec = (self.dest.coord - self.source.coord) / self.length
        #max speed -> 전부 1500
        self.max_speed = max_speed
        #edge 위에 존재하는 OHT 리스트
        self.OHTs = []  # Use a set to track OHTs
        
        self.count = 0  # OHT가 이 Edge에 진입한 횟수
        # self.speeds = []  # 최근 time_window 동안의 속도 저장
        self.avg_speed = max_speed  # time_window 내 평균 속도
        self.entry_exit_records = {} 
            
    def calculate_avg_speed(self, time_window, current_time):
        all_relevant_records = []
        total_time_covered = 0  # 기록된 총 시간
        total_distance_covered = 0  # 기록된 총 거리

        for oht_id, records in self.entry_exit_records.items():
            # time_window 내의 유효한 기록 필터링
            relevant_records = [
                (entry, exit) for entry, exit in records
                if exit is not None and entry >= current_time - time_window
            ]
            all_relevant_records.extend(relevant_records)

        # 기록된 각 구간의 속도와 이동 거리 계산
        for entry, exit in all_relevant_records:
            travel_time = exit - entry
            if travel_time > 0.01:  # 최소 이동 시간
                speed = self.length / travel_time
                total_time_covered += travel_time
                total_distance_covered += speed * travel_time

        # time_window에서 기록되지 않은 시간 간격 계산
        uncovered_time = time_window - total_time_covered
        if uncovered_time > 0:
            total_distance_covered += self.max_speed * uncovered_time

        # 전체 평균 속도 계산
        avg_speed = total_distance_covered / time_window
        return min(avg_speed, self.max_speed)
    
class port():
    def __init__(self, name, from_node, to_node, from_dist):
        #port name
        self.name = name
        #포트 위치 설정을 위한 from_node
        self.from_node = from_node
        #포트 to_node
        self.to_node = to_node
        #from_node로부터의 거리
        self.from_dist = from_dist
        #포트가 존재하는 edge
        self.edge = None

        
class OHT():
    def __init__(self, id, from_node, from_dist, speed, acc, rect, path):
        self.id = id #oht id
        self.from_node = from_node #oht 위치를 계산하기 위한 from_node
        self.from_dist = from_dist #from_node로부터의 거리
        
        self.path = path #oht가 움직일 경로, (Edge들의 List)
        self.path_to_start = [] #start port로 갈 때 경로
        self.path_to_end = [] #end port로 갈 때 경로
        self.edge = path.pop(0) if path else None #OHT가 위치한 edge
        
        self.pos = (
            self.from_node.coord + self.edge.unit_vec * self.from_dist
            if self.edge else self.from_node.coord
        ) #OHT의 위치 계산
        self.speed = speed #속도
        self.acc = acc #가속도
        self.rect = rect #충돌 감지 범위
        
        self.edge.OHTs.append(self) if self.edge else None #OHT가 위치한 edge의 OHT list에 self 추가 
        
        
        self.start_port = None #출발 포트
        self.end_port = None #도착 포트
        self.wait_time = 0 #loading / unloading시 기다리는 시간
        
        self.status = "IDLE" #STATUS, IDLE / TO_START / TO_END
    
    #위치 계산 method
    def cal_pos(self, time_step):
        self.from_dist = self.from_dist + self.speed * time_step + 1/2 * self.acc * time_step**2
        self.pos = self.from_node.coord + self.edge.unit_vec * self.from_dist if self.edge != None else self.from_node.coord

    #move, 매 time step 마다 실행            
    def move(self, time_step, current_time):
        #node 위에만 있을 떄?? 이거 사실 잘 모르겠구 에러 나는거 해결하려고 이거저거하다가 넣었습니다
        
        if self.status == 'ON_REMOVED':
            self.speed = 0
            self.acc = 0
            return
        
        if not self.edge:
            # print('no edge', self.edge)
            self.speed = 0
            self.acc = 0
            if len(self.path) != 0:
                self.from_node.OHT = None
            return

        #end port 도착하면 wait time 동안 기다리기
        if self.wait_time > 0:
            self.wait_time -= time_step
            if self.wait_time <= 0:  # 대기 시간이 끝나면
                self.wait_time = 0
                if self.status == 'STOP_AT_START':
                    self.status = 'TO_END'
                elif self.status == 'STOP_AT_END':
                    self.status = 'IDLE'
            return  # 대기 중이므로 이동하지 않음
        
        #충돌 감지
        self.col_check(time_step)
        
        # ori_dist_1 = copy.copy(self.from_dist)
        
        #다음 스텝에 움직인 거리 계산
        # self.from_dist = self.from_dist + self.speed * time_step + 1/2 * self.acc * time_step**2
        
        #만약 도착했다면 멈추기 
        if self.is_arrived():
            self.arrive()
            return
            
        #From_dist가 edge의 length보다 길어지면 다음 엣지로 업데이트    
        while (self.from_dist > self.edge.length):
            
            self.from_node = self.edge.dest #from_node를 Edge dest로 업데이트

            self.from_dist = self.from_dist - self.edge.length #from_dist도 업데이트
            
            if self.edge:
                try:
                     #원래 엣지에서 현재 OHT 제거
                    if len(self.path) > 0:
                # before_edge = copy.copy(self.edge)
                
                        exit_record = self.edge.entry_exit_records.get(self.id, [])
                        if exit_record and exit_record[-1][1] is None:
                            exit_record[-1] = (exit_record[-1][0], current_time)
                        self.edge.entry_exit_records[self.id] = exit_record
                                            
                        self.edge.OHTs.remove(self)
                        
                        self.edge = self.path.pop(0) #다음 엣지로 업데이트
                        
                        if self not in self.edge.OHTs:
                            self.edge.OHTs.append(self) #다음 엣지 안에 OHT 추가
                            self.edge.count += 1
                            if self.id not in self.edge.entry_exit_records:
                                self.edge.entry_exit_records[self.id] = []
                                self.edge.entry_exit_records[self.id].append((current_time, None))
                            
                    else:
                        self.speed = 0
                        self.acc = 0
                        self.from_dist = self.edge.length
                        self.from_node = self.edge.source
                        return
                except:
                    print('update error : ', self.edge.source.id)

  
            if self.is_arrived():
                self.arrive()
                return
            

        if self.is_arrived():
            self.arrive()
            return
            
        #가속도 고려해서 스피드 계산
        self.speed = min(max(self.speed + self.acc * time_step, 0), self.edge.max_speed)
        
    def is_arrived(self):

        #start_port 혹은 end port에 도착했는지 확인, OHT와 port가 같은 엣지, 같은 from_node에 있는지, OHT의 from_dist가 port의 From_dist보다 커지는지
        if self.status == "TO_START":
            return (self.start_port 
                    and self.start_port.from_node == self.from_node and self.start_port.to_node == self.edge.dest
                    and self.from_dist >= self.start_port.from_dist)
        elif self.status == "TO_END":
            return (self.end_port 
                    and self.end_port.from_node == self.from_node and self.end_port.to_node == self.edge.dest
                    and self.from_dist >= self.end_port.from_dist)
        
    def arrive(self):
        # print('arrived at', self.pos, self.from_node.id)
        # self.from_dist = self.end_port.from_dist
        # self.pos = self.from_node.coord + self.edge.unit_vec * self.end_port.from_dist
        # self.speed = 0
        # self.acc = 0
        # self.wait_time = 500
        # self.end_port = None
        # print('arrived rail : ', self.edge.OHTs, self in self.edge.OHTs)
        
        #도착하면 port위치로 OHT를 고정하고, 속도, 가속도 0으로 정지, wait_time 5초 주기
        if self.status == "TO_START":
            # print(f"OHT {self.id} arrived at start port: {self.start_port.name}")
            self.from_dist = self.start_port.from_dist
            self.pos = self.from_node.coord + self.edge.unit_vec * self.start_port.from_dist
            self.speed = 0
            self.acc = 0
            self.wait_time = 5
            
            self.status = "STOP_AT_START"
            self.path = self.path_to_end  # 경로를 end_port로 변경
            self.path_to_start = []  # start_port 경로 초기화
            self.start_port = None

        elif self.status == "TO_END":
            
            self.from_dist = self.end_port.from_dist
            self.pos = self.from_node.coord + self.edge.unit_vec * self.end_port.from_dist
            self.speed = 0
            self.acc = 0
            self.wait_time = 5
            self.end_port = None
            # print(f"OHT {self.id} arrived at end port: {self.end_port.name}")
            self.status = "STOP_AT_END"
            self.path = []
            self.path_to_end = []  # end_port 경로 초기화

        else:
            print(f"OHT {self.id} is idle.")

    
    #충돌 감지
    def col_check(self, time_step):
        OHTs = self.edge.OHTs
        
        #같은 edge상에 있는 OHT들과 거리 비교
        for oht in OHTs:
            if oht is not self:  # 자신과의 비교를 피함
                dist_diff = oht.from_dist - self.from_dist
                if 0 < dist_diff < self.rect:  # rect 길이보다 가까워지면
                    # 가속도를 줄여 충돌 방지
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
                
        if self.edge.dest.OHT is not None:
            dist_diff = self.edge.length - self.from_dist
            if 0 < dist_diff < self.rect:  # rect 길이보다 가까워지면
                # 가속도를 줄여 충돌 방지
                self.acc = -self.speed/time_step # 속도 감소 또는 정지
                return
         
         #다음 엣지에 있는 OHT들 중 제일 마지막에 있는 친구와 거리 비교       
        if len(self.path) > 0:
            next_edge = self.path[0]
            try:
                last_oht = next_edge.OHTs[-1]
                rem_diff = self.edge.length - self.from_dist
                dist_diff = last_oht.from_dist + rem_diff
                if 0 < dist_diff < self.rect:
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
            except:
                pass
        
        #다음 노드에서 들어오는 엣지들 상에 있는 OHT들과 충돌 비교 (intersection이 무조건 최대 2개임)    
        incoming_edges = [
            edge for edge in self.edge.dest.incoming_edges
            if edge != self.edge # Edges leading to the same destination node
        ]
        
        if len(incoming_edges) == 1: #만약 intersection이 있따면 (다른 edge가 있다면)
            rem_diff = self.edge.length - self.from_dist #현재 자기 자신의 엣지 상에서 남은 거리 계산
            try:
                other_oht = incoming_edges[0].OHTs[0]
                other_diff= other_oht.edge.length - other_oht.from_dist #다른 엣지 위 제일 앞에 있는 OHT의 남은 거리 계산
                dist_diff = rem_diff + other_diff #두 OHT간의 거리 계산
                if 0 < dist_diff < 3 * self.rect and rem_diff > other_diff: #더 가까운 OHT가 먼저 움직이도록, 나머지는 정지. 3*rect로 잡은 이유는 그래야 좀 더 미리 멈춰서
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
                elif rem_diff == other_diff and self.id > other_oht.id: #혹시나 거리가 같다면 OHT id가 더 빠른 아이가 움직이도록
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
            except:
                pass
        if len(self.path) > 0:
            outgoing_edges = [
                edge for edge in self.edge.dest.outgoing_edges
                if edge != self.path[0] # Edges leading to the same destination node
            ]
        
            if len(outgoing_edges) == 1: #만약 intersection이 있따면 (다른 edge가 있다면)
                rem_diff = self.edge.length - self.from_dist #현재 자기 자신의 엣지 상에서 남은 거리 계산
                try:
                    other_oht = outgoing_edges[0].OHTs[0]
                    other_diff= other_oht.from_dist #다른 엣지 위 제일 앞에 있는 OHT의 남은 거리 계산
                    dist_diff = rem_diff + other_diff #두 OHT간의 거리 계산
                    if 0 < dist_diff <  self.rect: #더 가까운 OHT가 먼저 움직이도록, 나머지는 정지. 3*rect로 잡은 이유는 그래야 좀 더 미리 멈춰서
                        self.acc = -self.speed/time_step # 속도 감소 또는 정지
                        return
                except:
                    pass
        
        #충돌 위험이 없다면 다시 원래 max speed로 가속
        self.acc = (self.edge.max_speed - self.speed) / time_step
        

class AMHS:
    def __init__(self, nodes, edges, ports, num_OHTs, max_jobs, job_list = [], oht_list = []):
        """
        AMHS 초기화.
        - nodes: node 클래스 객체 리스트
        - edges: edge 클래스 객체 리스트
        - num_OHTs: 초기 OHT 수
        - max_jobs: 작업 큐의 최대 크기
        """
        # self.graph = nx.DiGraph()
        # self.original_graph=nx.DiGraph()
        
        self.graph = nk.Graph(directed=True, weighted=True)  # Directed weighted graph
        
        self.nodes = nodes  # node 객체 리스트
        for node in nodes:
            node.OHT = None
        self.edges = edges  # edge 객체 리스트
        for edge in edges:
            edge.OHTs = []
            edge.count = 0
            edge.avg_speed = edge.max_speed
        self.ports = ports #port list
        for p in ports:
            p.edge = self.get_edge(p.from_node, p.to_node)
        
        self.OHTs = []
        
        if job_list != []:
            self.job_queue = [[self.get_port(q[0]), self.get_port(q[1])] for q in job_list]
        else:
            self.job_queue = []
        self.max_jobs = max_jobs
        
        self.node_id_map = {}
        
        self.time_step=0.01
        
                
        self.simulation_running = False  # 시뮬레이션 상태 관리
        self.stop_simulation_event = threading.Event()  # 스레드 이벤트 처리

        # 그래프 생성
        self._create_graph()
        
        self.original_graph = nk.Graph(self.graph)

        # 초기 OHT 배치
        if oht_list != []:
            self.set_initial_OHTs(oht_list)
        else:
            self.initialize_OHTs(num_OHTs)
        
        self.apsp = nk.distance.APSP(self.graph)
        self.apsp.run()
        
        self.original_apsp = nk.distance.APSP(self.original_graph)
        self.original_apsp.run()
        


    def _create_graph(self):
        """NetworkX 그래프를 nodes와 edges로 생성."""
        for i, node in enumerate(self.nodes):
            self.graph.addNode()  # Add node to the graph
            self.node_id_map[node.id] = i
        
        for edge in self.edges:
            u = self.node_id_map[edge.source.id]  # Get index for source node
            v = self.node_id_map[edge.dest.id]    # Get index for destination node
            self.graph.addEdge(u, v, edge.length)
            # edge.dest.incoming_edges.append(edge) #각 node마다 incoming edge 추가
            # edge.source.outgoing_edges.append(edge)

    def initialize_OHTs(self, num_OHTs):
        available_nodes = self.nodes.copy()
        
        for i in range(num_OHTs):
            # 노드 중에서 랜덤 선택 (노드 소진 시 다시 전체에서 랜덤)
            if not available_nodes:
                available_nodes = self.nodes.copy()  # 노드 리스트 재생성
            
            start_node = random.choice(available_nodes)
            available_nodes.remove(start_node)  # 선택한 노드는 제거
            
            oht = OHT(
                id=i,
                from_node=start_node,
                from_dist=0,
                speed=0,
                acc=0,
                rect=1000,  # 충돌 판정 거리
                path=[],
            )
            start_node.OHT = oht
            self.OHTs.append(oht)
            
    def set_initial_OHTs(self, oht_list):
        i = 0
        for start_node in oht_list:  
            start_node = self.get_node(start_node)
            oht = OHT(
                id=i,
                from_node=start_node,
                from_dist=0,
                speed=0,
                acc=0,
                rect=1000,  # 충돌 판정 거리
                path=[],
            )
            start_node.OHT = oht
            self.OHTs.append(oht)
            i = i+1
    
    def set_oht(self, oht_origin, oht_new):
        oht_origin.from_node = self.get_node(oht_new['from_node'])
        oht_origin.from_dist = oht_new['from_dist']
        oht_origin.edge = self.get_edge(oht_new['source'], oht_new['dest'])
        if oht_origin.edge:
            if oht_origin not in oht_origin.edge.OHTs:
                oht_origin.edge.OHTs.append(oht_origin)
        try:
            if not self.graph.hasEdge(self.node_id_map[oht_new['source']], self.node_id_map[oht_new['dest']]):
                oht_origin.speed = 0
                oht_origin.acc = 0
                oht_origin.status = 'ON_REMOVED'
                oht_origin.cal_pos(self.time_step)
                return
        except:
            print('is here?', oht_new['source'])
  
            
        oht_origin.speed = oht_new['speed']
        oht_origin.status = oht_new['status']
        
        if oht_origin.status == 'ON_REMOVED':
            if oht_origin.start_port:
                oht_origin.status = "TO_START"
            elif oht_origin.end_port:
                oht_origin.status = "TO_END"
            else:
                oht_origin.status = "IDLE"
                
        oht_origin.cal_pos(self.time_step)
             
        
        oht_origin.start_port = self.get_port(oht_new['startPort']) if oht_new['startPort'] else None
        oht_origin.end_port = self.get_port(oht_new['endPort']) if oht_new['endPort'] else None
        oht_origin.wait_time = oht_new['wait_time']
                
        if oht_origin.start_port != None:
            if oht_origin.edge:
                path_edges_to_start = self.get_path_edges(oht_origin.edge.dest.id, oht_origin.start_port.to_node.id)
                path_edges_to_start = self._validate_path(path_edges_to_start)

            else:
                path_edges_to_start = self.get_path_edges(oht_origin.from_node.id, oht_origin.start_port.to_node.id)
                path_edges_to_start = self._validate_path(path_edges_to_start)
            oht_origin.path_to_start = path_edges_to_start[:]
        
        # elif oht_origin.status == "TO_END":
        if oht_origin.end_port != None:
            if oht_origin.status == "TO_END" or oht_origin.status == 'STOP_AT_START':
                if oht_origin.edge:
                    path_edges_to_end = self.get_path_edges(oht_origin.edge.dest.id, oht_origin.end_port.to_node.id)
                    path_edges_to_end = self._validate_path(path_edges_to_end)
                else:
                    path_edges_to_end = self.get_path_edges(oht_origin.from_node.id, oht_origin.end_port.to_node.id)
                    path_edges_to_end = self._validate_path(path_edges_to_end)
            else:
                path_edges_to_end = self.get_path_edges(oht_origin.start_port.to_node.id, oht_origin.end_port.to_node.id)

            oht_origin.path_to_end = path_edges_to_end
        
        if oht_origin.status == "TO_START":
            oht_origin.path = path_edges_to_start[:]
        elif oht_origin.status == "TO_END" or oht_origin.status == "STOP_AT_START":
            oht_origin.path = path_edges_to_end[:]
        else:
            oht_origin.path = []

    def modi_edge(self, source, dest, oht_positions, is_removed):
        if is_removed:
            removed_edge = self.get_edge(source, dest)
            try:       
                self.graph.removeEdge(self.node_id_map[source], self.node_id_map[dest])
                self.apsp = nk.distance.APSP(self.graph)
                self.apsp.run()
            except:
                print(source, dest)
        else:
            edge_to_restore = next((e for e in self.edges if e.source.id == source and e.dest.id == dest), None)
            if edge_to_restore:
                self.graph.addEdge(self.node_id_map[source], self.node_id_map[dest], edge_to_restore.length)
                self.apsp = nk.distance.APSP(self.graph)
                self.apsp.run()
            else:
                print(f"Rail {source} -> {dest} not found in original edges.")
        
    
    def reinitialize_simul(self, oht_positions, edge_data):
        for edge in self.edges:
            edge.OHTs.clear()
            edge.entry_exit_records.clear()
        
        for oht in oht_positions:
            oht_in_amhs = self.get_oht(oht['id'])
            self.set_oht(oht_in_amhs, oht)
            
        edge_map = {f"{edge.source.id}-{edge.dest.id}": edge for edge in self.edges}
    
        for edge_info in edge_data:
            edge_key = f"{edge_info['from']}-{edge_info['to']}"
            if edge_key in edge_map:
                edge = edge_map[edge_key]
                edge.count = edge_info.get('count', 0)
                edge.avg_speed = edge_info.get('avg_speed', 0)
            
        for edge in self.edges:
            edge.OHTs.sort(key = lambda oht : -oht.from_dist)
    
    def generate_job(self):
        """모든 OHT가 Job을 갖도록 작업 생성."""
        for _ in range(50):
            start_port = random.choice(self.ports)
            end_port = random.choice(self.ports)
            while start_port == end_port:  # 시작/목적 포트가 같지 않도록 보장
                end_port = random.choice(self.ports)
            self.job_queue.append((start_port, end_port))
            

    def assign_jobs(self):
        """모든 OHT가 Job을 갖도록 작업 할당."""
        closest_oht = None
        closest_dist = 1e100
        for oht in self.OHTs:
            if not oht.path and self.job_queue and oht.status == 'IDLE':  # OHT가 Idle 상태이고 Job Queue가 비어있지 않은 경우
                start_port, end_port = self.job_queue.pop(0)
                dist = self.get_path_distance(oht.from_node.id, start_port.from_node.id)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_oht = oht
        
        if closest_oht:
            oht = closest_oht
            oht.start_port = start_port
            # oht.start_port = next((port for port in self.ports if port.name == 'DIE0137_Port3'), None)
            oht.end_port = end_port
            oht.status = "TO_START"

            # # Start로 이동하는 경로
            if oht.edge:
                path_edges_to_start = self.get_path_edges(oht.edge.dest.id, start_port.to_node.id)
            else:
                path_edges_to_start = self.get_path_edges(oht.from_node.id, start_port.to_node.id)
            oht.path_to_start = path_edges_to_start[:]
            
            # # End로 이동하는 경로
            # start_edge = [self.get_edge(start_port.from_node.id, start_port.to_node.id)]
            path_edges_to_end = self.get_path_edges(start_port.to_node.id, end_port.to_node.id)
            oht.path_to_end = path_edges_to_end

            # # 전체 경로를 OHT에 설정
            oht.path = path_edges_to_start[:]
            
            # print(oht.path)

            # Assign the first edge in the path to the OHT's edge
            if oht.path:
                if not oht.edge:
                    oht.edge = oht.path.pop(0)
                    oht.from_node.OHT = None
                    if oht not in oht.edge.OHTs:
                        oht.edge.OHTs.append(oht)
                        
    # def update_edge_metrics(self, current_time, time_window):
    #     for edge in self.edges:
    #         edge.avg_speed = edge.calculate_avg_speed(time_window, current_time)


    def get_node(self, node_id):
        """node id로 node 객체 반환."""
        return next((node for node in self.nodes if node.id == node_id), None)

    def get_edge(self, source_id, dest_id):
        """source와 dest ID로 edge 객체 반환."""
        return next(
            (e for e in self.edges if e.source.id == source_id and e.dest.id == dest_id), 
            None
        )
        
    def get_path_distance(self, source_id, dest_id):
        try:
            source_idx = self.node_id_map[source_id]
            dest_idx = self.node_id_map[dest_id]

            
            dist = self.apsp.getDistance(source_idx, dest_idx)      
            return dist
        except:
            try:
                source_idx = self.node_id_map[source_id]
                dest_idx = self.node_id_map[dest_id]            

                dist = self.original_apsp.getDistance(source_idx, dest_idx)                
                return dist
            except:
                print(f"No path found even in original_graph for {source_id} -> {dest_id}.")
                return float('inf')
        

    def get_path_edges(self, source_id, dest_id):
        """source와 dest ID로 최단 경로 에지 리스트 반환."""
        try:
            source_idx = self.node_id_map[source_id]
            dest_idx = self.node_id_map[dest_id]
            
            dijkstra = nk.distance.Dijkstra(self.graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
            dijkstra.run()

            path = dijkstra.getPath(dest_idx)
            
            return [
                self.get_edge(self.nodes[path[i]].id, self.nodes[path[i+1]].id)
                for i in range(len(path) - 1)
            ]
        except:
            try:
                source_idx = self.node_id_map[source_id]
                dest_idx = self.node_id_map[dest_id]
                
                dijkstra = nk.distance.Dijkstra(self.original_graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
                dijkstra.run()

                path = dijkstra.getPath(dest_idx)
                
                return [
                    self.get_edge(self.nodes[path[i]].id, self.nodes[path[i+1]].id)
                    for i in range(len(path) - 1)
                ]
            except:
                print(f"No path found even in original_graph for {source_id} -> {dest_id}.")
                return []
    
    def get_oht(self, oht_id):
        return next((oht for oht in self.OHTs if oht.id == oht_id), None)
    
    def get_port(self, port_name):
        return next((port for port in self.ports if port.name == port_name), None)

    def _validate_path(self, path):
        """
        현재 경로에 지워진 edge가 포함된 경우, 해당 edge 직전까지만 반환합니다.

        Parameters:
            path (list): 현재 경로 (edge 객체 리스트)

        Returns:
            list: 수정된 경로 (지워진 edge 직전까지만 포함)
        """
        valid_path = []
        for edge in path:
            if self.graph.hasEdge(self.node_id_map[edge.source.id], self.node_id_map[edge.dest.id]):  # edge가 그래프에 존재하는 경우
                valid_path.append(edge)
            else:
                break  # 지워진 edge를 발견하면 직전까지만 반환
        return valid_path    
        
    def start_simulation(self, current_time, max_time = 4000, time_step = 0.01):
        
        
        count = 0
        edge_metrics_cache = {}  # Cache for edge metrics to track changes
        

        while current_time < max_time:
            if count % 5 == 0:
                self.generate_job()
                self.assign_jobs()

            # Move all OHTs
            oht_positions = []
            
            # self.OHTs = self.update_oht_positions(self.OHTs, current_time, time_step)
            for oht in self.OHTs:
                oht.move(time_step, current_time)
                
            ray.put(self.edges)

            self.edges = self.update_edge_metrics(current_time, time_window=500)
            
            for oht in self.OHTs:
                oht.cal_pos(time_step)
                

            if count % 5 == 0:
                for oht in self.OHTs:
                    oht_positions.append({
                        'id': oht.id,
                        'x': oht.pos[0],
                        'y': oht.pos[1],
                        'source': oht.edge.source.id if oht.edge else None,
                        'dest': oht.edge.dest.id if oht.edge else None,
                        'speed': oht.speed,
                        'status': oht.status,
                        'startPort': oht.start_port.name if oht.start_port else None,
                        'endPort': oht.end_port.name if oht.end_port else None,
                        'from_node': oht.from_node.id if oht.from_node else None,
                        'from_dist': oht.from_dist,
                        'wait_time': oht.wait_time
                    })

                updated_edges = []
                for edge in self.edges:
                    key = f"{edge.source.id}-{edge.dest.id}"
                    new_metrics = {"count": edge.count, "avg_speed": edge.avg_speed}
                    if edge_metrics_cache.get(key) != new_metrics:
                        edge_metrics_cache[key] = new_metrics
                        updated_edges.append({
                            "from": edge.source.id,
                            "to": edge.dest.id,
                            **new_metrics
                        })

                payload = {
                    'time': current_time,
                    'oht_positions': oht_positions,
                    'edges': updated_edges
                }
                
                compressed_payload = compress_data(payload)

            # Increment time
            current_time += time_step
            count += 1

        self.simulation_running = False
        print('Simulation ended')
        


import random
import json
import time
import threading
import math
import pandas as pd
import io



# Load the layout data
with open('./flask_backend/fab_oht_layout_2nd.json') as f:
    layout_data = json.load(f)



nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]
edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1500  # Default speed = 0.01
    )
    for rail in layout_data['rails']
]

ports = [
    port(
        name = p['name'], 
        from_node = next((node for node in nodes if node.id == p['from_node'])),
        to_node = next((node for node in nodes if node.id == p['to_node'])),
        from_dist = p['distance']
    )
    for p in layout_data['ports']
]


ray.init(num_cpus=4,ignore_reinit_error=True)  # Ray 클러스터 초기화
max_time = 4000
    
amhs = AMHS(nodes=nodes, edges=edges, ports=ports, num_OHTs=500, max_jobs=1000)

ray.put(amhs.nodes)
# ray.put(amhs)
# ray.get(amhs.start_simulation.remote(current_time=0, max_time=4000))
ray.shutdown()  # Ray 클러스터 종료

2025-01-23 04:03:56,090	INFO worker.py:1821 -- Started a local Ray instance.


In [6]:
import numpy as np
import random
import networkit as nk
import json
import threading
import ray

class node():
    def __init__(self, id, coord):
        self.id = id
        self.coord = np.array(coord)
        self.incoming_edges = []
        self.outgoing_edges = []
        self.OHT = None        
    
    def add_incoming_edge(self, edge):
        self.incoming_edges.append(edge)

    def add_outgoing_edge(self, edge):
        self.outgoing_edges.append(edge)
        
class edge():
    def __init__(self, source, dest, length, max_speed):
        self.id = None
        self.source = source
        self.dest = dest
        self.length = length
        self.unit_vec = (self.dest.coord - self.source.coord) / self.length
        self.max_speed = max_speed
        self.OHTs = []  # Use a set to track OHTs        
        self.count = 0  # OHT가 이 Edge에 진입한 횟수
        self.avg_speed = max_speed  # time_window 내 평균 속도
        self.entry_exit_records = {}        
        source.add_incoming_edge(self)
        dest.add_outgoing_edge(self)
        
class port():
    def __init__(self, name, from_node, to_node, from_dist):
        #port name
        self.name = name
        self.from_node = from_node
        self.to_node = to_node
        self.from_dist = from_dist
        self.edge = None


class AMHS:
    def __init__(self, nodes, edges, ports, num_OHTs, max_jobs, job_list = [], oht_list = []):        
        self.graph = nk.Graph(directed=True, weighted=True)  # Directed weighted graph
        
        self.nodes = nodes  # node 객체 리스트
        for node in nodes:
            node.OHT = None
        self.edges = edges  # edge 객체 리스트
        for edge in edges:
            edge.OHTs = []
            edge.count = 0
            edge.avg_speed = edge.max_speed
        self.ports = ports #port list
        for p in ports:
            p.edge = self.get_edge(p.from_node, p.to_node)
        self.OHTs = []
        if job_list != []:
            self.job_queue = [[self.get_port(q[0]), self.get_port(q[1])] for q in job_list]
        else:
            self.job_queue = []
        self.max_jobs = max_jobs
        
        self.node_id_map = {}
        
        self.edge_id_map = {}
        
        self.time_step=0.01
        
                
        # self.simulation_running = False  # 시뮬레이션 상태 관리
        # self.stop_simulation_event = threading.Event()  # 스레드 이벤트 처리

        # 그래프 생성
        # self.create_graph()
        
        # self.original_graph = nk.Graph(self.graph)

        # self.apsp = nk.distance.APSP(self.graph)
        # self.apsp.run()
        
        # self.original_apsp = nk.distance.APSP(self.original_graph)
        # self.original_apsp.run()


    def create_graph(self):
        """NetworkX 그래프를 nodes와 edges로 생성."""
        for i, node in enumerate(self.nodes):
            self.graph.addNode()  # Add node to the graph
            self.node_id_map[node.id] = i
        
        for i, edge in enumerate(self.edges):
            self.edge_id_map[i] = edge
            edge.id = i
        
        for edge in self.edges:
            u = self.node_id_map[edge.source.id]  # Get index for source node
            v = self.node_id_map[edge.dest.id]    # Get index for destination node
            self.graph.addEdge(u, v, edge.length)
            

    def get_node(self, node_id):
        """node id로 node 객체 반환."""
        return next((node for node in self.nodes if node.id == node_id), None)

    def get_edge(self, source_id, dest_id):
        """source와 dest ID로 edge 객체 반환."""
        return next(
            (e for e in self.edges if e.source.id == source_id and e.dest.id == dest_id), 
            None
        )
        
    def get_path_distance(self, source_id, dest_id):
        try:
            source_idx = self.node_id_map[source_id]
            dest_idx = self.node_id_map[dest_id]
            
            dist = self.apsp.getDistance(source_idx, dest_idx)      
            return dist
        except:
            try:
                source_idx = self.node_id_map[source_id]
                dest_idx = self.node_id_map[dest_id]            

                dist = self.original_apsp.getDistance(source_idx, dest_idx)                
                return dist
            except:
                print(f"No path found even in original_graph for {source_id} -> {dest_id}.")
                return float('inf')
        

    def get_path_edges(self, source_id, dest_id):
        """source와 dest ID로 최단 경로 에지 리스트 반환."""
        try:
            source_idx = self.node_id_map[source_id]
            dest_idx = self.node_id_map[dest_id]
            
            dijkstra = nk.distance.Dijkstra(self.graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
            dijkstra.run()

            path = dijkstra.getPath(dest_idx)
            
            return [
                self.get_edge(self.nodes[path[i]].id, self.nodes[path[i+1]].id)
                for i in range(len(path) - 1)
            ]
        except:
            try:
                source_idx = self.node_id_map[source_id]
                dest_idx = self.node_id_map[dest_id]
                
                dijkstra = nk.distance.Dijkstra(self.original_graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
                dijkstra.run()

                path = dijkstra.getPath(dest_idx)
                
                return [
                    self.get_edge(self.nodes[path[i]].id, self.nodes[path[i+1]].id)
                    for i in range(len(path) - 1)
                ]
            except:
                print(f"No path found even in original_graph for {source_id} -> {dest_id}.")
                return []
    
    def get_oht(self, oht_id):
        return next((oht for oht in self.OHTs if oht.id == oht_id), None)
    
    def get_port(self, port_name):
        return next((port for port in self.ports if port.name == port_name), None)

    def _validate_path(self, path):
        """
        현재 경로에 지워진 edge가 포함된 경우, 해당 edge 직전까지만 반환합니다.

        Parameters:
            path (list): 현재 경로 (edge 객체 리스트)

        Returns:
            list: 수정된 경로 (지워진 edge 직전까지만 포함)
        """
        valid_path = []
        for edge in path:
            if self.graph.hasEdge(self.node_id_map[edge.source.id], self.node_id_map[edge.dest.id]):  # edge가 그래프에 존재하는 경우
                valid_path.append(edge)
            else:
                break  # 지워진 edge를 발견하면 직전까지만 반환
        return valid_path

        
import random
import json
import time
import threading
import math
import pandas as pd
import io



# Load the layout data
with open('./flask_backend/fab_oht_layout_2nd.json') as f:
    layout_data = json.load(f)


nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]
edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1500  # Default speed = 0.01
    )
    for rail in layout_data['rails']
]

ports = [
    port(
        name = p['name'], 
        from_node = next((node for node in nodes if node.id == p['from_node'])),
        to_node = next((node for node in nodes if node.id == p['to_node'])),
        from_dist = p['distance']
    )
    for p in layout_data['ports']
]

if __name__ == "__main__":
    ray.init(num_cpus=4,ignore_reinit_error=True)  # Ray 클러스터 초기화
    max_time = 4000
        
    amhs = AMHS(nodes=nodes, edges=edges, ports=ports, num_OHTs=500, max_jobs=1000)
    
    ray.put(amhs.nodes)
    
    ray.shutdown()  # Ray 클러스터 종료

2025-01-23 07:51:02,241	INFO worker.py:1654 -- Calling ray.init() again after it has already been called.


PicklingError: Could not pickle object as excessively deep recursion required.

In [28]:
import ray
import random
import json
import math
import numpy as np

# 클래스 정의 (node, edge, OHT 등 포함)
class node():
    def __init__(self, id, coord):
        self.id = id
        self.coord = np.array(coord)
        self.incoming_edges = []
        self.outgoing_edges = []
        self.OHT = None

    def add_incoming_edge(self, edge):
        self.incoming_edges.append(edge)

    def add_outgoing_edge(self, edge):
        self.outgoing_edges.append(edge)

class edge():
    def __init__(self, source, dest, length, max_speed):
        self.source = source
        self.dest = dest
        self.length = length
        self.max_speed = max_speed
        self.OHTs = []
        
        self.update_edge_references()

    def update_edge_references(self):
        self.source.add_incoming_edge(self)
        self.dest.add_outgoing_edge(self)

class OHT():
    def __init__(self, id, from_node, from_dist, speed, acc, rect, path):
        self.id = id
        self.from_node = from_node
        self.from_dist = from_dist
        self.speed = speed
        self.acc = acc
        self.rect = rect
        self.path = path
        self.pos = self.from_node.coord
        self.status = "IDLE"

# AMHS 클래스 정의 (간단화)
class AMHS:
    def __init__(self, nodes, edges, num_OHTs):
        self.nodes = nodes
        self.edges = edges
        self.OHTs = []
        self.initialize_OHTs(num_OHTs)
    
    def initialize_OHTs(self, num_OHTs):
        available_nodes = self.nodes.copy()
        for i in range(num_OHTs):
            start_node = random.choice(available_nodes)
            available_nodes.remove(start_node)
            oht = OHT(id=i, from_node=start_node, from_dist=0, speed=0, acc=0, rect=1000, path=[])
            start_node.OHT = oht
            self.OHTs.append(oht)


# def test_ray_put():
#     nodes = [node(i, [i, i]) for i in range(500)]  # 5개의 노드
#     edges = [edge(nodes[i], nodes[(i+1) % 5], 10, 1500) for i in range(500)]  # 5개의 엣지 (순환 구조)
#     amhs = AMHS(nodes, edges, num_OHTs=500)

#     # Ray에 데이터를 직렬화하여 전달
#     ray.put(amhs.nodes)
#     ray.put(amhs.edges)
#     ray.put(amhs.OHTs)

#     print("Node Data, Edge Data, OHT Data put into Ray!")

# Ray 클러스터 초기화
ray.init(ignore_reinit_error=True)

# nodes = [node(i, [i, i]) for i in range(500)]  # 5개의 노드
edges = [edge(nodes[i], nodes[(i+1) % 5], 1023923.1241, 1500) for i in range(500)]  # 5개의 엣지 (순환 구조)

# Load the layout data
with open('./flask_backend/fab_oht_layout_2nd.json') as f:
    layout_data = json.load(f)


nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]
edges = [edge(random.choice(nodes), random.choice(nodes), 1023923.1241, 1500) for i in range(3000)]
# edges = [
#     edge(
#         source=next(node for node in nodes if node.id == rail['from']),
#         dest=next(node for node in nodes if node.id == rail['to']),
#         length=math.sqrt(
#             (next(node for node in nodes if node.id == rail['to']).coord[0] -
#              next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
#             (next(node for node in nodes if node.id == rail['to']).coord[1] -
#              next(node for node in nodes if node.id == rail['from']).coord[1])**2
#         ),
#         max_speed=1500  # Default speed = 0.01
#     )
#     for rail in layout_data['rails']
# ]

amhs = AMHS(nodes, edges, num_OHTs=500)

# Ray에 데이터를 직렬화하여 전달
ray.put(amhs.nodes)
ray.put(amhs.edges)
# ray.put(amhs.OHTs)

# Ray 테스트
# test_ray_put()

# Ray 클러스터 종료
ray.shutdown()





# ray.init(num_cpus=4,ignore_reinit_error=True)  # Ray 클러스터 초기화
# max_time = 4000
    
# amhs = AMHS(nodes=nodes, edges=edges, num_OHTs=500)

# ray.put(amhs.nodes)

# ray.shutdown()  # Ray 클러스터 종료

2025-01-23 08:11:33,126	INFO worker.py:1821 -- Started a local Ray instance.


PicklingError: Could not pickle object as excessively deep recursion required.

In [2]:
nodes = [node(i, [i, i]) for i in range(500)]  # 5개의 노드
edges = [edge(nodes[i], nodes[(i+1) % 5], 10, 1500) for i in range(500)]  # 5개의 엣지 (순환 구조)

In [6]:
edges[0].source

In [7]:
nodes

In [8]:
with open('./flask_backend/fab_oht_layout_2nd.json') as f:
    layout_data = json.load(f)


nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]
edges = [
    edge(
        source=next(node for node in nodes if node.id == rail['from']),
        dest=next(node for node in nodes if node.id == rail['to']),
        length=math.sqrt(
            (next(node for node in nodes if node.id == rail['to']).coord[0] -
             next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
            (next(node for node in nodes if node.id == rail['to']).coord[1] -
             next(node for node in nodes if node.id == rail['from']).coord[1])**2
        ),
        max_speed=1500  # Default speed = 0.01
    )
    for rail in layout_data['rails']
]


In [9]:
nodes

 ...]

In [11]:
edges[0].source

In [15]:
import ray
import json
import math
import numpy as np

# Node 클래스 직렬화 가능하도록 개선
class node():
    def __init__(self, id, coord):
        self.id = id
        self.coord = np.array(coord)
        self.incoming_edges = []
        self.outgoing_edges = []
        self.OHT = None

    def add_incoming_edge(self, edge):
        self.incoming_edges.append(edge)

    def add_outgoing_edge(self, edge):
        self.outgoing_edges.append(edge)

class edge():
    def __init__(self, source, dest, length, max_speed):
        self.source = source
        self.dest = dest
        self.length = length
        self.max_speed = max_speed
        self.OHTs = []
        self.update_edge_references()

    def update_edge_references(self):
        self.source.add_incoming_edge(self)
        self.dest.add_outgoing_edge(self)

# 테스트 함수
def test_ray_put_with_layout_data():
    # Layout Data 로드 (파일 경로는 실제 경로로 바꿔야 함)
    with open('./flask_backend/fab_oht_layout_2nd.json') as f:
        layout_data = json.load(f)

    # Node 객체 생성
    nodes = [node(n['id'], [n['x'], n['y']]) for n in layout_data['nodes']]
    print(f"Loaded {len(nodes)} nodes")

    # Edge 객체 생성
    edges = [
        edge(
            source=next(node for node in nodes if node.id == rail['from']),
            dest=next(node for node in nodes if node.id == rail['to']),
            length=math.sqrt(
                (next(node for node in nodes if node.id == rail['to']).coord[0] -
                 next(node for node in nodes if node.id == rail['from']).coord[0])**2 +
                (next(node for node in nodes if node.id == rail['to']).coord[1] -
                 next(node for node in nodes if node.id == rail['from']).coord[1])**2
            ),
            max_speed=1500
        )
        for rail in layout_data['rails']
    ]
    print(f"Loaded {len(edges)} edges")

    # Ray에 직렬화 가능한 형태로 데이터 전송
    ray.put(nodes)
    ray.put(edges)

    print("Nodes and edges data have been put into Ray!")

# Ray 클러스터 초기화
ray.init(ignore_reinit_error=True)

# Ray 테스트
test_ray_put_with_layout_data()

# Ray 클러스터 종료
ray.shutdown()


2025-01-23 08:07:20,658	INFO worker.py:1821 -- Started a local Ray instance.


Loaded 2858 nodes
Loaded 3424 edges


PicklingError: Could not pickle object as excessively deep recursion required.

In [34]:
import numpy as np
import random
import networkit as nk
import threading
import gzip
import base64

def compress_data(data):
    json_data = json.dumps(data).encode('utf-8')
    compressed_data = gzip.compress(json_data)
    return base64.b64encode(compressed_data).decode('utf-8')


#node 클래스
class node():
    def __init__(self, id, coord):
        #node id
        self.id = id
        #node 좌표 (2차원)
        self.coord = np.array(coord)
        #노드로 들어오는 edge들 => intersection 쪽 충돌 감지 위해 필요
        self.incoming_edges = []
        self.outgoing_edges = []
        self.OHT = None
        
class edge():
    def __init__(self, source, dest, length, max_speed):
        #Edge source
        self.source = source
        #edge target(destination)
        self.dest = dest
        #edge length
        self.length = length
        #edge의 방향 벡터
        self.unit_vec = (self.dest.coord - self.source.coord) / self.length
        #max speed -> 전부 1500
        self.max_speed = max_speed
        #edge 위에 존재하는 OHT 리스트
        self.OHTs = []  # Use a set to track OHTs
        
        self.count = 0  # OHT가 이 Edge에 진입한 횟수
        # self.speeds = []  # 최근 time_window 동안의 속도 저장
        self.avg_speed = max_speed  # time_window 내 평균 속도
        self.entry_exit_records = {} 
            
    def calculate_avg_speed(self, time_window, current_time):
        all_relevant_records = []
        total_time_covered = 0  # 기록된 총 시간
        total_distance_covered = 0  # 기록된 총 거리

        for oht_id, records in self.entry_exit_records.items():
            # time_window 내의 유효한 기록 필터링
            relevant_records = [
                (entry, exit) for entry, exit in records
                if exit is not None and entry >= current_time - time_window
            ]
            all_relevant_records.extend(relevant_records)

        # 기록된 각 구간의 속도와 이동 거리 계산
        for entry, exit in all_relevant_records:
            travel_time = exit - entry
            if travel_time > 0.01:  # 최소 이동 시간
                speed = self.length / travel_time
                total_time_covered += travel_time
                total_distance_covered += speed * travel_time

        # time_window에서 기록되지 않은 시간 간격 계산
        uncovered_time = time_window - total_time_covered
        if uncovered_time > 0:
            total_distance_covered += self.max_speed * uncovered_time

        # 전체 평균 속도 계산
        avg_speed = total_distance_covered / time_window
        return min(avg_speed, self.max_speed)
    
class port():
    def __init__(self, name, from_node, to_node, from_dist):
        #port name
        self.name = name
        #포트 위치 설정을 위한 from_node
        self.from_node = from_node
        #포트 to_node
        self.to_node = to_node
        #from_node로부터의 거리
        self.from_dist = from_dist
        #포트가 존재하는 edge
        self.edge = None

        
class OHT():
    def __init__(self, id, from_node, from_dist, speed, acc, rect, path):
        self.id = id #oht id
        self.from_node = from_node #oht 위치를 계산하기 위한 from_node
        self.from_dist = from_dist #from_node로부터의 거리
        
        self.path = path #oht가 움직일 경로, (Edge들의 List)
        self.path_to_start = [] #start port로 갈 때 경로
        self.path_to_end = [] #end port로 갈 때 경로
        self.edge = path.pop(0) if path else None #OHT가 위치한 edge
        
        self.pos = (
            self.from_node.coord + self.edge.unit_vec * self.from_dist
            if self.edge else self.from_node.coord
        ) #OHT의 위치 계산
        self.speed = speed #속도
        self.acc = acc #가속도
        self.rect = rect #충돌 감지 범위
        
        self.edge.OHTs.append(self) if self.edge else None #OHT가 위치한 edge의 OHT list에 self 추가 
        
        
        self.start_port = None #출발 포트
        self.end_port = None #도착 포트
        self.wait_time = 0 #loading / unloading시 기다리는 시간
        
        self.status = "IDLE" #STATUS, IDLE / TO_START / TO_END
    
    #위치 계산 method
    def cal_pos(self, time_step):
        self.from_dist = self.from_dist + self.speed * time_step + 1/2 * self.acc * time_step**2
        self.pos = self.from_node.coord + self.edge.unit_vec * self.from_dist if self.edge != None else self.from_node.coord

    #move, 매 time step 마다 실행            
    def move(self, time_step, current_time):
        #node 위에만 있을 떄?? 이거 사실 잘 모르겠구 에러 나는거 해결하려고 이거저거하다가 넣었습니다
        
        if self.status == 'ON_REMOVED':
            self.speed = 0
            self.acc = 0
            return
        
        if not self.edge:
            # print('no edge', self.edge)
            self.speed = 0
            self.acc = 0
            if len(self.path) != 0:
                self.from_node.OHT = None
            return

        #end port 도착하면 wait time 동안 기다리기
        if self.wait_time > 0:
            self.wait_time -= time_step
            if self.wait_time <= 0:  # 대기 시간이 끝나면
                self.wait_time = 0
                if self.status == 'STOP_AT_START':
                    self.status = 'TO_END'
                elif self.status == 'STOP_AT_END':
                    self.status = 'IDLE'
            return  # 대기 중이므로 이동하지 않음
        
        #충돌 감지
        self.col_check(time_step)
        
        # ori_dist_1 = copy.copy(self.from_dist)
        
        #다음 스텝에 움직인 거리 계산
        # self.from_dist = self.from_dist + self.speed * time_step + 1/2 * self.acc * time_step**2
        
        #만약 도착했다면 멈추기 
        if self.is_arrived():
            self.arrive()
            return
            
        #From_dist가 edge의 length보다 길어지면 다음 엣지로 업데이트    
        while (self.from_dist > self.edge.length):
            
            self.from_node = self.edge.dest #from_node를 Edge dest로 업데이트

            self.from_dist = self.from_dist - self.edge.length #from_dist도 업데이트
            
            if self.edge:
                try:
                     #원래 엣지에서 현재 OHT 제거
                    if len(self.path) > 0:
                # before_edge = copy.copy(self.edge)
                
                        exit_record = self.edge.entry_exit_records.get(self.id, [])
                        if exit_record and exit_record[-1][1] is None:
                            exit_record[-1] = (exit_record[-1][0], current_time)
                        self.edge.entry_exit_records[self.id] = exit_record
                                            
                        self.edge.OHTs.remove(self)
                        
                        self.edge = self.path.pop(0) #다음 엣지로 업데이트
                        
                        if self not in self.edge.OHTs:
                            self.edge.OHTs.append(self) #다음 엣지 안에 OHT 추가
                            self.edge.count += 1
                            if self.id not in self.edge.entry_exit_records:
                                self.edge.entry_exit_records[self.id] = []
                                self.edge.entry_exit_records[self.id].append((current_time, None))
                            
                    else:
                        self.speed = 0
                        self.acc = 0
                        self.from_dist = self.edge.length
                        self.from_node = self.edge.source
                        return
                except:
                    print('update error : ', self.edge.source.id)

  
            if self.is_arrived():
                self.arrive()
                return
            

        if self.is_arrived():
            self.arrive()
            return
            
        #가속도 고려해서 스피드 계산
        self.speed = min(max(self.speed + self.acc * time_step, 0), self.edge.max_speed)
        
    def is_arrived(self):

        #start_port 혹은 end port에 도착했는지 확인, OHT와 port가 같은 엣지, 같은 from_node에 있는지, OHT의 from_dist가 port의 From_dist보다 커지는지
        if self.status == "TO_START":
            return (self.start_port 
                    and self.start_port.from_node == self.from_node and self.start_port.to_node == self.edge.dest
                    and self.from_dist >= self.start_port.from_dist)
        elif self.status == "TO_END":
            return (self.end_port 
                    and self.end_port.from_node == self.from_node and self.end_port.to_node == self.edge.dest
                    and self.from_dist >= self.end_port.from_dist)
        
    def arrive(self):
        # print('arrived at', self.pos, self.from_node.id)
        # self.from_dist = self.end_port.from_dist
        # self.pos = self.from_node.coord + self.edge.unit_vec * self.end_port.from_dist
        # self.speed = 0
        # self.acc = 0
        # self.wait_time = 500
        # self.end_port = None
        # print('arrived rail : ', self.edge.OHTs, self in self.edge.OHTs)
        
        #도착하면 port위치로 OHT를 고정하고, 속도, 가속도 0으로 정지, wait_time 5초 주기
        if self.status == "TO_START":
            # print(f"OHT {self.id} arrived at start port: {self.start_port.name}")
            self.from_dist = self.start_port.from_dist
            self.pos = self.from_node.coord + self.edge.unit_vec * self.start_port.from_dist
            self.speed = 0
            self.acc = 0
            self.wait_time = 5
            
            self.status = "STOP_AT_START"
            self.path = self.path_to_end  # 경로를 end_port로 변경
            self.path_to_start = []  # start_port 경로 초기화
            self.start_port = None

        elif self.status == "TO_END":
            
            self.from_dist = self.end_port.from_dist
            self.pos = self.from_node.coord + self.edge.unit_vec * self.end_port.from_dist
            self.speed = 0
            self.acc = 0
            self.wait_time = 5
            self.end_port = None
            # print(f"OHT {self.id} arrived at end port: {self.end_port.name}")
            self.status = "STOP_AT_END"
            self.path = []
            self.path_to_end = []  # end_port 경로 초기화

        else:
            print(f"OHT {self.id} is idle.")

    
    #충돌 감지
    def col_check(self, time_step):
        OHTs = self.edge.OHTs
        
        #같은 edge상에 있는 OHT들과 거리 비교
        for oht in OHTs:
            if oht is not self:  # 자신과의 비교를 피함
                dist_diff = oht.from_dist - self.from_dist
                if 0 < dist_diff < self.rect:  # rect 길이보다 가까워지면
                    # 가속도를 줄여 충돌 방지
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
                
        if self.edge.dest.OHT is not None:
            dist_diff = self.edge.length - self.from_dist
            if 0 < dist_diff < self.rect:  # rect 길이보다 가까워지면
                # 가속도를 줄여 충돌 방지
                self.acc = -self.speed/time_step # 속도 감소 또는 정지
                return
         
         #다음 엣지에 있는 OHT들 중 제일 마지막에 있는 친구와 거리 비교       
        if len(self.path) > 0:
            next_edge = self.path[0]
            try:
                last_oht = next_edge.OHTs[-1]
                rem_diff = self.edge.length - self.from_dist
                dist_diff = last_oht.from_dist + rem_diff
                if 0 < dist_diff < self.rect:
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
            except:
                pass
        
        #다음 노드에서 들어오는 엣지들 상에 있는 OHT들과 충돌 비교 (intersection이 무조건 최대 2개임)    
        incoming_edges = [
            edge for edge in self.edge.dest.incoming_edges
            if edge != self.edge # Edges leading to the same destination node
        ]
        
        if len(incoming_edges) == 1: #만약 intersection이 있따면 (다른 edge가 있다면)
            rem_diff = self.edge.length - self.from_dist #현재 자기 자신의 엣지 상에서 남은 거리 계산
            try:
                other_oht = incoming_edges[0].OHTs[0]
                other_diff= other_oht.edge.length - other_oht.from_dist #다른 엣지 위 제일 앞에 있는 OHT의 남은 거리 계산
                dist_diff = rem_diff + other_diff #두 OHT간의 거리 계산
                if 0 < dist_diff < 3 * self.rect and rem_diff > other_diff: #더 가까운 OHT가 먼저 움직이도록, 나머지는 정지. 3*rect로 잡은 이유는 그래야 좀 더 미리 멈춰서
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
                elif rem_diff == other_diff and self.id > other_oht.id: #혹시나 거리가 같다면 OHT id가 더 빠른 아이가 움직이도록
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
            except:
                pass
        if len(self.path) > 0:
            outgoing_edges = [
                edge for edge in self.edge.dest.outgoing_edges
                if edge != self.path[0] # Edges leading to the same destination node
            ]
        
            if len(outgoing_edges) == 1: #만약 intersection이 있따면 (다른 edge가 있다면)
                rem_diff = self.edge.length - self.from_dist #현재 자기 자신의 엣지 상에서 남은 거리 계산
                try:
                    other_oht = outgoing_edges[0].OHTs[0]
                    other_diff= other_oht.from_dist #다른 엣지 위 제일 앞에 있는 OHT의 남은 거리 계산
                    dist_diff = rem_diff + other_diff #두 OHT간의 거리 계산
                    if 0 < dist_diff <  self.rect: #더 가까운 OHT가 먼저 움직이도록, 나머지는 정지. 3*rect로 잡은 이유는 그래야 좀 더 미리 멈춰서
                        self.acc = -self.speed/time_step # 속도 감소 또는 정지
                        return
                except:
                    pass
        
        #충돌 위험이 없다면 다시 원래 max speed로 가속
        self.acc = (self.edge.max_speed - self.speed) / time_step
        
class AMHS:
    def __init__(self, nodes, edges, ports, num_OHTs, max_jobs, job_list = [], oht_list = []):
        """
        AMHS 초기화.
        - nodes: node 클래스 객체 리스트
        - edges: edge 클래스 객체 리스트
        - num_OHTs: 초기 OHT 수
        - max_jobs: 작업 큐의 최대 크기
        """
        # self.graph = nx.DiGraph()
        # self.original_graph=nx.DiGraph()
        
        self.graph = nk.Graph(directed=True, weighted=True)  # Directed weighted graph
        
        self.nodes = nodes  # node 객체 리스트
        for node in nodes:
            node.OHT = None
        self.edges = edges  # edge 객체 리스트
        for edge in edges:
            edge.OHTs = []
            edge.count = 0
            edge.avg_speed = edge.max_speed
        self.ports = ports #port list
        for p in ports:
            p.edge = self.get_edge(p.from_node, p.to_node)
        self.OHTs = []
        if job_list != []:
            self.job_queue = [[self.get_port(q[0]), self.get_port(q[1])] for q in job_list]
        else:
            self.job_queue = []
        self.max_jobs = max_jobs
        
        self.node_id_map = {}
        
        self.time_step=0.01
        
                
        self.simulation_running = False  # 시뮬레이션 상태 관리
        self.stop_simulation_event = threading.Event()  # 스레드 이벤트 처리

        # 그래프 생성
        self._create_graph()
        
        self.original_graph = nk.Graph(self.graph)

        # 초기 OHT 배치
        if oht_list != []:
            self.set_initial_OHTs(oht_list)
        else:
            self.initialize_OHTs(num_OHTs)
        
        self.apsp = nk.distance.APSP(self.graph)
        self.apsp.run()
        
        self.original_apsp = nk.distance.APSP(self.original_graph)
        self.original_apsp.run()
        


    def _create_graph(self):
        """NetworkX 그래프를 nodes와 edges로 생성."""
        for i, node in enumerate(self.nodes):
            self.graph.addNode()  # Add node to the graph
            self.node_id_map[node.id] = i
        
        for edge in self.edges:
            u = self.node_id_map[edge.source.id]  # Get index for source node
            v = self.node_id_map[edge.dest.id]    # Get index for destination node
            self.graph.addEdge(u, v, edge.length)
            edge.dest.incoming_edges.append(edge) #각 node마다 incoming edge 추가
            edge.source.outgoing_edges.append(edge)

    def initialize_OHTs(self, num_OHTs):
        available_nodes = self.nodes.copy()
        
        for i in range(num_OHTs):
            # 노드 중에서 랜덤 선택 (노드 소진 시 다시 전체에서 랜덤)
            if not available_nodes:
                available_nodes = self.nodes.copy()  # 노드 리스트 재생성
            
            start_node = random.choice(available_nodes)
            available_nodes.remove(start_node)  # 선택한 노드는 제거
            
            oht = OHT(
                id=i,
                from_node=start_node,
                from_dist=0,
                speed=0,
                acc=0,
                rect=1000,  # 충돌 판정 거리
                path=[],
            )
            start_node.OHT = oht
            self.OHTs.append(oht)
            
    def set_initial_OHTs(self, oht_list):
        i = 0
        for start_node in oht_list:  
            start_node = self.get_node(start_node)
            oht = OHT(
                id=i,
                from_node=start_node,
                from_dist=0,
                speed=0,
                acc=0,
                rect=1000,  # 충돌 판정 거리
                path=[],
            )
            start_node.OHT = oht
            self.OHTs.append(oht)
            i = i+1
    
    def set_oht(self, oht_origin, oht_new):
        oht_origin.from_node = self.get_node(oht_new['from_node'])
        oht_origin.from_dist = oht_new['from_dist']
        oht_origin.edge = self.get_edge(oht_new['source'], oht_new['dest'])
        if oht_origin.edge:
            if oht_origin not in oht_origin.edge.OHTs:
                oht_origin.edge.OHTs.append(oht_origin)
        try:
            if not self.graph.hasEdge(self.node_id_map[oht_new['source']], self.node_id_map[oht_new['dest']]):
                oht_origin.speed = 0
                oht_origin.acc = 0
                oht_origin.status = 'ON_REMOVED'
                oht_origin.cal_pos(self.time_step)
                return
        except:
            print('is here?', oht_new['source'])
  
            
        oht_origin.speed = oht_new['speed']
        oht_origin.status = oht_new['status']
        
        if oht_origin.status == 'ON_REMOVED':
            if oht_origin.start_port:
                oht_origin.status = "TO_START"
            elif oht_origin.end_port:
                oht_origin.status = "TO_END"
            else:
                oht_origin.status = "IDLE"
                
        oht_origin.cal_pos(self.time_step)
             
        
        oht_origin.start_port = self.get_port(oht_new['startPort']) if oht_new['startPort'] else None
        oht_origin.end_port = self.get_port(oht_new['endPort']) if oht_new['endPort'] else None
        oht_origin.wait_time = oht_new['wait_time']
                
        if oht_origin.start_port != None:
            if oht_origin.edge:
                path_edges_to_start = self.get_path_edges(oht_origin.edge.dest.id, oht_origin.start_port.to_node.id)
                path_edges_to_start = self._validate_path(path_edges_to_start)

            else:
                path_edges_to_start = self.get_path_edges(oht_origin.from_node.id, oht_origin.start_port.to_node.id)
                path_edges_to_start = self._validate_path(path_edges_to_start)
            oht_origin.path_to_start = path_edges_to_start[:]
        
        # elif oht_origin.status == "TO_END":
        if oht_origin.end_port != None:
            if oht_origin.status == "TO_END" or oht_origin.status == 'STOP_AT_START':
                if oht_origin.edge:
                    path_edges_to_end = self.get_path_edges(oht_origin.edge.dest.id, oht_origin.end_port.to_node.id)
                    path_edges_to_end = self._validate_path(path_edges_to_end)
                else:
                    path_edges_to_end = self.get_path_edges(oht_origin.from_node.id, oht_origin.end_port.to_node.id)
                    path_edges_to_end = self._validate_path(path_edges_to_end)
            else:
                path_edges_to_end = self.get_path_edges(oht_origin.start_port.to_node.id, oht_origin.end_port.to_node.id)

            oht_origin.path_to_end = path_edges_to_end
        
        if oht_origin.status == "TO_START":
            oht_origin.path = path_edges_to_start[:]
        elif oht_origin.status == "TO_END" or oht_origin.status == "STOP_AT_START":
            oht_origin.path = path_edges_to_end[:]
        else:
            oht_origin.path = []

    def modi_edge(self, source, dest, oht_positions, is_removed):
        if is_removed:
            removed_edge = self.get_edge(source, dest)
            try:       
                self.graph.removeEdge(self.node_id_map[source], self.node_id_map[dest])
                self.apsp = nk.distance.APSP(self.graph)
                self.apsp.run()
            except:
                print(source, dest)
        else:
            edge_to_restore = next((e for e in self.edges if e.source.id == source and e.dest.id == dest), None)
            if edge_to_restore:
                self.graph.addEdge(self.node_id_map[source], self.node_id_map[dest], edge_to_restore.length)
                self.apsp = nk.distance.APSP(self.graph)
                self.apsp.run()
            else:
                print(f"Rail {source} -> {dest} not found in original edges.")
        
    
    def reinitialize_simul(self, oht_positions, edge_data):
        for edge in self.edges:
            edge.OHTs.clear()
            edge.entry_exit_records.clear()
        
        for oht in oht_positions:
            oht_in_amhs = self.get_oht(oht['id'])
            self.set_oht(oht_in_amhs, oht)
            
        edge_map = {f"{edge.source.id}-{edge.dest.id}": edge for edge in self.edges}
    
        for edge_info in edge_data:
            edge_key = f"{edge_info['from']}-{edge_info['to']}"
            if edge_key in edge_map:
                edge = edge_map[edge_key]
                edge.count = edge_info.get('count', 0)
                edge.avg_speed = edge_info.get('avg_speed', 0)
            
        for edge in self.edges:
            edge.OHTs.sort(key = lambda oht : -oht.from_dist)
    
    def generate_job(self):
        """모든 OHT가 Job을 갖도록 작업 생성."""
        for _ in range(50):
            start_port = random.choice(self.ports)
            end_port = random.choice(self.ports)
            while start_port == end_port:  # 시작/목적 포트가 같지 않도록 보장
                end_port = random.choice(self.ports)
            self.job_queue.append((start_port, end_port))
            

    def assign_jobs(self):
        """모든 OHT가 Job을 갖도록 작업 할당."""
        closest_oht = None
        closest_dist = 1e100
        for oht in self.OHTs:
            if not oht.path and self.job_queue and oht.status == 'IDLE':  # OHT가 Idle 상태이고 Job Queue가 비어있지 않은 경우
                start_port, end_port = self.job_queue.pop(0)
                dist = self.get_path_distance(oht.from_node.id, start_port.from_node.id)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_oht = oht
        
        if closest_oht:
            oht = closest_oht
            oht.start_port = start_port
            # oht.start_port = next((port for port in self.ports if port.name == 'DIE0137_Port3'), None)
            oht.end_port = end_port
            oht.status = "TO_START"

            # # Start로 이동하는 경로
            if oht.edge:
                path_edges_to_start = self.get_path_edges(oht.edge.dest.id, start_port.to_node.id)
            else:
                path_edges_to_start = self.get_path_edges(oht.from_node.id, start_port.to_node.id)
            oht.path_to_start = path_edges_to_start[:]
            
            # # End로 이동하는 경로
            # start_edge = [self.get_edge(start_port.from_node.id, start_port.to_node.id)]
            path_edges_to_end = self.get_path_edges(start_port.to_node.id, end_port.to_node.id)
            oht.path_to_end = path_edges_to_end

            # # 전체 경로를 OHT에 설정
            oht.path = path_edges_to_start[:]
            
            # print(oht.path)

            # Assign the first edge in the path to the OHT's edge
            if oht.path:
                if not oht.edge:
                    oht.edge = oht.path.pop(0)
                    oht.from_node.OHT = None
                    if oht not in oht.edge.OHTs:
                        oht.edge.OHTs.append(oht)
                        
    def update_edge_metrics(self, current_time, time_window):
        for edge in self.edges:
            edge.avg_speed = edge.calculate_avg_speed(time_window, current_time)


    def get_node(self, node_id):
        """node id로 node 객체 반환."""
        return next((node for node in self.nodes if node.id == node_id), None)

    def get_edge(self, source_id, dest_id):
        """source와 dest ID로 edge 객체 반환."""
        return next(
            (e for e in self.edges if e.source.id == source_id and e.dest.id == dest_id), 
            None
        )
        
    def get_path_distance(self, source_id, dest_id):
        try:
            source_idx = self.node_id_map[source_id]
            dest_idx = self.node_id_map[dest_id]
            
            # dijkstra = nk.distance.Dijkstra(self.graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
            # dijkstra.run()
            # dist = dijkstra.distance(dest_idx)
            
            dist = self.apsp.getDistance(source_idx, dest_idx)      
            return dist
        except:
            try:
                source_idx = self.node_id_map[source_id]
                dest_idx = self.node_id_map[dest_id]
                
                
                
                # dijkstra = nk.distance.Dijkstra(self.original_graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
                # dijkstra.run()

                # dist = dijkstra.distance(dest_idx)
            

                dist = self.original_apsp.getDistance(source_idx, dest_idx)                
                return dist
            except:
                print(f"No path found even in original_graph for {source_id} -> {dest_id}.")
                return float('inf')
        

    def get_path_edges(self, source_id, dest_id):
        """source와 dest ID로 최단 경로 에지 리스트 반환."""
        try:
            source_idx = self.node_id_map[source_id]
            dest_idx = self.node_id_map[dest_id]
            
            dijkstra = nk.distance.Dijkstra(self.graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
            dijkstra.run()

            path = dijkstra.getPath(dest_idx)
            
            return [
                self.get_edge(self.nodes[path[i]].id, self.nodes[path[i+1]].id)
                for i in range(len(path) - 1)
            ]
        except:
            try:
                source_idx = self.node_id_map[source_id]
                dest_idx = self.node_id_map[dest_id]
                
                dijkstra = nk.distance.Dijkstra(self.original_graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
                dijkstra.run()

                path = dijkstra.getPath(dest_idx)
                
                return [
                    self.get_edge(self.nodes[path[i]].id, self.nodes[path[i+1]].id)
                    for i in range(len(path) - 1)
                ]
            except:
                print(f"No path found even in original_graph for {source_id} -> {dest_id}.")
                return []
    
    def get_oht(self, oht_id):
        return next((oht for oht in self.OHTs if oht.id == oht_id), None)
    
    def get_port(self, port_name):
        return next((port for port in self.ports if port.name == port_name), None)

    def _validate_path(self, path):
        """
        현재 경로에 지워진 edge가 포함된 경우, 해당 edge 직전까지만 반환합니다.

        Parameters:
            path (list): 현재 경로 (edge 객체 리스트)

        Returns:
            list: 수정된 경로 (지워진 edge 직전까지만 포함)
        """
        valid_path = []
        for edge in path:
            if self.graph.hasEdge(self.node_id_map[edge.source.id], self.node_id_map[edge.dest.id]):  # edge가 그래프에 존재하는 경우
                valid_path.append(edge)
            else:
                break  # 지워진 edge를 발견하면 직전까지만 반환
        return valid_path
    
    
        
    def start_simulation(self, socketio, current_time, max_time = 4000, time_step = 0.01):
        """시뮬레이션 시작"""        
        if self.simulation_running:
            print("Simulation is already running. Stopping the current simulation...")
            self.stop_simulation_event.set()
            while self.simulation_running:
                socketio.sleep(0.01)  # Wait for the current simulation to stop
            return
        
        self.simulation_running = True
        self.stop_simulation_event.clear()
        
        count = 0
        edge_metrics_cache = {}  # Cache for edge metrics to track changes

        while current_time < max_time:
            if self.stop_simulation_event.is_set():
                break
            
            if count % 5 == 0:
                self.generate_job()
                self.assign_jobs()

            # Move all OHTs
            oht_positions = []
            for oht in self.OHTs:
                oht.move(time_step, current_time)

            self.update_edge_metrics(current_time, time_window=500)
            
            for oht in self.OHTs:
                oht.cal_pos(time_step)

            if count % 5 == 0:
                for oht in self.OHTs:
                    oht_positions.append({
                        'id': oht.id,
                        'x': oht.pos[0],
                        'y': oht.pos[1],
                        'source': oht.edge.source.id if oht.edge else None,
                        'dest': oht.edge.dest.id if oht.edge else None,
                        'speed': oht.speed,
                        'status': oht.status,
                        'startPort': oht.start_port.name if oht.start_port else None,
                        'endPort': oht.end_port.name if oht.end_port else None,
                        'from_node': oht.from_node.id if oht.from_node else None,
                        'from_dist': oht.from_dist,
                        'wait_time': oht.wait_time
                    })

                updated_edges = []
                for edge in self.edges:
                    key = f"{edge.source.id}-{edge.dest.id}"
                    new_metrics = {"count": edge.count, "avg_speed": edge.avg_speed}
                    if edge_metrics_cache.get(key) != new_metrics:
                        edge_metrics_cache[key] = new_metrics
                        updated_edges.append({
                            "from": edge.source.id,
                            "to": edge.dest.id,
                            **new_metrics
                        })

                payload = {
                    'time': current_time,
                    'oht_positions': oht_positions,
                    'edges': updated_edges
                }

                compressed_payload = compress_data(payload)
                socketio.emit('updateOHT', {'data': compressed_payload})

            # Increment time
            current_time += time_step
            count += 1
            socketio.sleep(0.00001)

        self.simulation_running = False
        print('Simulation ended')
        
        
    
    def accelerate_simul(self, socketio, current_time, max_time = 4000, time_step = 0.01):
        """시뮬레이션 시작"""        
        if self.simulation_running:
            print("Simulation is already running. Stopping the current simulation...")
            self.stop_simulation_event.set()
            while self.simulation_running:
                socketio.sleep(0.01)  # Wait for the current simulation to stop
            return
        
        self.simulation_running = True
        self.stop_simulation_event.clear()
        
        _current_time = 0
        
        count = 0

        while _current_time < current_time:
            if self.stop_simulation_event.is_set():
                break
            
            if count % 5 == 0:
                self.generate_job()
                self.assign_jobs()
            
            if count % 10 == 0:
                print(_current_time)

            # Move all OHTs
            
            for oht in self.OHTs:
                oht.move(time_step, _current_time)

            self.update_edge_metrics(_current_time, time_window=500)
            
            for oht in self.OHTs:
                oht.cal_pos(time_step)

            # Increment time
            _current_time += time_step
            count += 1
            
        
        edge_metrics_cache = {} 
            
        
        while _current_time < max_time:
            if self.stop_simulation_event.is_set():
                break
            
            if count % 5 == 0:
                self.generate_job()
                self.assign_jobs()

            # Move all OHTs
            oht_positions = []
            for oht in self.OHTs:
                oht.move(time_step, _current_time)

            self.update_edge_metrics(_current_time, time_window=500)
            
            for oht in self.OHTs:
                oht.cal_pos(time_step)

            if count % 5 == 0:
                for oht in self.OHTs:
                    oht_positions.append({
                        'id': oht.id,
                        'x': oht.pos[0],
                        'y': oht.pos[1],
                        'source': oht.edge.source.id if oht.edge else None,
                        'dest': oht.edge.dest.id if oht.edge else None,
                        'speed': oht.speed,
                        'status': oht.status,
                        'startPort': oht.start_port.name if oht.start_port else None,
                        'endPort': oht.end_port.name if oht.end_port else None,
                        'from_node': oht.from_node.id if oht.from_node else None,
                        'from_dist': oht.from_dist,
                        'wait_time': oht.wait_time
                    })

                updated_edges = []
                for edge in self.edges:
                    key = f"{edge.source.id}-{edge.dest.id}"
                    new_metrics = {"count": edge.count, "avg_speed": edge.avg_speed}
                    if edge_metrics_cache.get(key) != new_metrics:
                        edge_metrics_cache[key] = new_metrics
                        updated_edges.append({
                            "from": edge.source.id,
                            "to": edge.dest.id,
                            **new_metrics
                        })

                payload = {
                    'time': current_time,
                    'oht_positions': oht_positions,
                    'edges': updated_edges
                }

                compressed_payload = compress_data(payload)
                socketio.emit('updateOHT', {'data': compressed_payload})

            # Increment time
            current_time += time_step
            count += 1
            socketio.sleep(0.00001)

        self.simulation_running = False
        print('Simulation ended')

2025-01-23 08:22:07,294	INFO worker.py:1821 -- Started a local Ray instance.


In [ ]:
#import libraries
import numpy as np
import random
import networkit as nk
import json
import threading
import gzip
import base64

def compress_data(data):
    json_data = json.dumps(data).encode('utf-8')
    compressed_data = gzip.compress(json_data)
    return base64.b64encode(compressed_data).decode('utf-8')


#node 클래스
class node():
    def __init__(self, id, coord):
        #node id
        self.id = id
        #node 좌표 (2차원)
        self.coord = np.array(coord)
        #노드로 들어오는 edge들 => intersection 쪽 충돌 감지 위해 필요
        self.incoming_edges = []
        self.outgoing_edges = []
        self.OHT = None
        
class edge():
    def __init__(self, source, dest, length, max_speed):
        #Edge source
        self.source = source
        #edge target(destination)
        self.dest = dest
        #edge length
        self.length = length
        #edge의 방향 벡터
        self.unit_vec = (self.dest.coord - self.source.coord) / self.length
        #max speed -> 전부 1500
        self.max_speed = max_speed
        #edge 위에 존재하는 OHT 리스트
        self.OHTs = []  # Use a set to track OHTs
        
        self.count = 0  # OHT가 이 Edge에 진입한 횟수
        # self.speeds = []  # 최근 time_window 동안의 속도 저장
        self.avg_speed = max_speed  # time_window 내 평균 속도
        self.entry_exit_records = {} 
            
    def calculate_avg_speed(self, time_window, current_time):
        all_relevant_records = []
        total_time_covered = 0  # 기록된 총 시간
        total_distance_covered = 0  # 기록된 총 거리

        for oht_id, records in self.entry_exit_records.items():
            # time_window 내의 유효한 기록 필터링
            relevant_records = [
                (entry, exit) for entry, exit in records
                if exit is not None and entry >= current_time - time_window
            ]
            all_relevant_records.extend(relevant_records)

        # 기록된 각 구간의 속도와 이동 거리 계산
        for entry, exit in all_relevant_records:
            travel_time = exit - entry
            if travel_time > 0.01:  # 최소 이동 시간
                speed = self.length / travel_time
                total_time_covered += travel_time
                total_distance_covered += speed * travel_time

        # time_window에서 기록되지 않은 시간 간격 계산
        uncovered_time = time_window - total_time_covered
        if uncovered_time > 0:
            total_distance_covered += self.max_speed * uncovered_time

        # 전체 평균 속도 계산
        avg_speed = total_distance_covered / time_window
        return min(avg_speed, self.max_speed)
    
class port():
    def __init__(self, name, from_node, to_node, from_dist):
        #port name
        self.name = name
        #포트 위치 설정을 위한 from_node
        self.from_node = from_node
        #포트 to_node
        self.to_node = to_node
        #from_node로부터의 거리
        self.from_dist = from_dist
        #포트가 존재하는 edge
        self.edge = None

        
class OHT():
    def __init__(self, id, from_node, from_dist, speed, acc, rect, path):
        self.id = id #oht id
        self.from_node = from_node #oht 위치를 계산하기 위한 from_node
        self.from_dist = from_dist #from_node로부터의 거리
        
        self.path = path #oht가 움직일 경로, (Edge들의 List)
        self.path_to_start = [] #start port로 갈 때 경로
        self.path_to_end = [] #end port로 갈 때 경로
        self.edge = path.pop(0) if path else None #OHT가 위치한 edge
        
        self.pos = (
            self.from_node.coord + self.edge.unit_vec * self.from_dist
            if self.edge else self.from_node.coord
        ) #OHT의 위치 계산
        self.speed = speed #속도
        self.acc = acc #가속도
        self.rect = rect #충돌 감지 범위
        
        self.edge.OHTs.append(self) if self.edge else None #OHT가 위치한 edge의 OHT list에 self 추가 
        
        
        self.start_port = None #출발 포트
        self.end_port = None #도착 포트
        self.wait_time = 0 #loading / unloading시 기다리는 시간
        
        self.status = "IDLE" #STATUS, IDLE / TO_START / TO_END
    
    #위치 계산 method
    def cal_pos(self, time_step):
        self.from_dist = self.from_dist + self.speed * time_step + 1/2 * self.acc * time_step**2
        self.pos = self.from_node.coord + self.edge.unit_vec * self.from_dist if self.edge != None else self.from_node.coord

    #move, 매 time step 마다 실행            
    def move(self, time_step, current_time):
        #node 위에만 있을 떄?? 이거 사실 잘 모르겠구 에러 나는거 해결하려고 이거저거하다가 넣었습니다
        
        if self.status == 'ON_REMOVED':
            self.speed = 0
            self.acc = 0
            return
        
        if not self.edge:
            # print('no edge', self.edge)
            self.speed = 0
            self.acc = 0
            if len(self.path) != 0:
                self.from_node.OHT = None
            return

        #end port 도착하면 wait time 동안 기다리기
        if self.wait_time > 0:
            self.wait_time -= time_step
            if self.wait_time <= 0:  # 대기 시간이 끝나면
                self.wait_time = 0
                if self.status == 'STOP_AT_START':
                    self.status = 'TO_END'
                elif self.status == 'STOP_AT_END':
                    self.status = 'IDLE'
            return  # 대기 중이므로 이동하지 않음
        
        #충돌 감지
        self.col_check(time_step)
        
        # ori_dist_1 = copy.copy(self.from_dist)
        
        #다음 스텝에 움직인 거리 계산
        # self.from_dist = self.from_dist + self.speed * time_step + 1/2 * self.acc * time_step**2
        
        #만약 도착했다면 멈추기 
        if self.is_arrived():
            self.arrive()
            return
            
        #From_dist가 edge의 length보다 길어지면 다음 엣지로 업데이트    
        while (self.from_dist > self.edge.length):
            
            self.from_node = self.edge.dest #from_node를 Edge dest로 업데이트

            self.from_dist = self.from_dist - self.edge.length #from_dist도 업데이트
            
            if self.edge:
                try:
                     #원래 엣지에서 현재 OHT 제거
                    if len(self.path) > 0:
                # before_edge = copy.copy(self.edge)
                
                        exit_record = self.edge.entry_exit_records.get(self.id, [])
                        if exit_record and exit_record[-1][1] is None:
                            exit_record[-1] = (exit_record[-1][0], current_time)
                        self.edge.entry_exit_records[self.id] = exit_record
                                            
                        self.edge.OHTs.remove(self)
                        
                        self.edge = self.path.pop(0) #다음 엣지로 업데이트
                        
                        if self not in self.edge.OHTs:
                            self.edge.OHTs.append(self) #다음 엣지 안에 OHT 추가
                            self.edge.count += 1
                            if self.id not in self.edge.entry_exit_records:
                                self.edge.entry_exit_records[self.id] = []
                                self.edge.entry_exit_records[self.id].append((current_time, None))
                            
                    else:
                        self.speed = 0
                        self.acc = 0
                        self.from_dist = self.edge.length
                        self.from_node = self.edge.source
                        return
                except:
                    print('update error : ', self.edge.source.id)

  
            if self.is_arrived():
                self.arrive()
                return
            

        if self.is_arrived():
            self.arrive()
            return
            
        #가속도 고려해서 스피드 계산
        self.speed = min(max(self.speed + self.acc * time_step, 0), self.edge.max_speed)
        
    def is_arrived(self):

        #start_port 혹은 end port에 도착했는지 확인, OHT와 port가 같은 엣지, 같은 from_node에 있는지, OHT의 from_dist가 port의 From_dist보다 커지는지
        if self.status == "TO_START":
            return (self.start_port 
                    and self.start_port.from_node == self.from_node and self.start_port.to_node == self.edge.dest
                    and self.from_dist >= self.start_port.from_dist)
        elif self.status == "TO_END":
            return (self.end_port 
                    and self.end_port.from_node == self.from_node and self.end_port.to_node == self.edge.dest
                    and self.from_dist >= self.end_port.from_dist)
        
    def arrive(self):
        # print('arrived at', self.pos, self.from_node.id)
        # self.from_dist = self.end_port.from_dist
        # self.pos = self.from_node.coord + self.edge.unit_vec * self.end_port.from_dist
        # self.speed = 0
        # self.acc = 0
        # self.wait_time = 500
        # self.end_port = None
        # print('arrived rail : ', self.edge.OHTs, self in self.edge.OHTs)
        
        #도착하면 port위치로 OHT를 고정하고, 속도, 가속도 0으로 정지, wait_time 5초 주기
        if self.status == "TO_START":
            # print(f"OHT {self.id} arrived at start port: {self.start_port.name}")
            self.from_dist = self.start_port.from_dist
            self.pos = self.from_node.coord + self.edge.unit_vec * self.start_port.from_dist
            self.speed = 0
            self.acc = 0
            self.wait_time = 5
            
            self.status = "STOP_AT_START"
            self.path = self.path_to_end  # 경로를 end_port로 변경
            self.path_to_start = []  # start_port 경로 초기화
            self.start_port = None

        elif self.status == "TO_END":
            
            self.from_dist = self.end_port.from_dist
            self.pos = self.from_node.coord + self.edge.unit_vec * self.end_port.from_dist
            self.speed = 0
            self.acc = 0
            self.wait_time = 5
            self.end_port = None
            # print(f"OHT {self.id} arrived at end port: {self.end_port.name}")
            self.status = "STOP_AT_END"
            self.path = []
            self.path_to_end = []  # end_port 경로 초기화

        else:
            print(f"OHT {self.id} is idle.")

    
    #충돌 감지
    def col_check(self, time_step):
        OHTs = self.edge.OHTs
        
        #같은 edge상에 있는 OHT들과 거리 비교
        for oht in OHTs:
            if oht is not self:  # 자신과의 비교를 피함
                dist_diff = oht.from_dist - self.from_dist
                if 0 < dist_diff < self.rect:  # rect 길이보다 가까워지면
                    # 가속도를 줄여 충돌 방지
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
                
        if self.edge.dest.OHT is not None:
            dist_diff = self.edge.length - self.from_dist
            if 0 < dist_diff < self.rect:  # rect 길이보다 가까워지면
                # 가속도를 줄여 충돌 방지
                self.acc = -self.speed/time_step # 속도 감소 또는 정지
                return
         
         #다음 엣지에 있는 OHT들 중 제일 마지막에 있는 친구와 거리 비교       
        if len(self.path) > 0:
            next_edge = self.path[0]
            try:
                last_oht = next_edge.OHTs[-1]
                rem_diff = self.edge.length - self.from_dist
                dist_diff = last_oht.from_dist + rem_diff
                if 0 < dist_diff < self.rect:
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
            except:
                pass
        
        #다음 노드에서 들어오는 엣지들 상에 있는 OHT들과 충돌 비교 (intersection이 무조건 최대 2개임)    
        incoming_edges = [
            edge for edge in self.edge.dest.incoming_edges
            if edge != self.edge # Edges leading to the same destination node
        ]
        
        if len(incoming_edges) == 1: #만약 intersection이 있따면 (다른 edge가 있다면)
            rem_diff = self.edge.length - self.from_dist #현재 자기 자신의 엣지 상에서 남은 거리 계산
            try:
                other_oht = incoming_edges[0].OHTs[0]
                other_diff= other_oht.edge.length - other_oht.from_dist #다른 엣지 위 제일 앞에 있는 OHT의 남은 거리 계산
                dist_diff = rem_diff + other_diff #두 OHT간의 거리 계산
                if 0 < dist_diff < 3 * self.rect and rem_diff > other_diff: #더 가까운 OHT가 먼저 움직이도록, 나머지는 정지. 3*rect로 잡은 이유는 그래야 좀 더 미리 멈춰서
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
                elif rem_diff == other_diff and self.id > other_oht.id: #혹시나 거리가 같다면 OHT id가 더 빠른 아이가 움직이도록
                    self.acc = -self.speed/time_step # 속도 감소 또는 정지
                    return
            except:
                pass
        if len(self.path) > 0:
            outgoing_edges = [
                edge for edge in self.edge.dest.outgoing_edges
                if edge != self.path[0] # Edges leading to the same destination node
            ]
        
            if len(outgoing_edges) == 1: #만약 intersection이 있따면 (다른 edge가 있다면)
                rem_diff = self.edge.length - self.from_dist #현재 자기 자신의 엣지 상에서 남은 거리 계산
                try:
                    other_oht = outgoing_edges[0].OHTs[0]
                    other_diff= other_oht.from_dist #다른 엣지 위 제일 앞에 있는 OHT의 남은 거리 계산
                    dist_diff = rem_diff + other_diff #두 OHT간의 거리 계산
                    if 0 < dist_diff <  self.rect: #더 가까운 OHT가 먼저 움직이도록, 나머지는 정지. 3*rect로 잡은 이유는 그래야 좀 더 미리 멈춰서
                        self.acc = -self.speed/time_step # 속도 감소 또는 정지
                        return
                except:
                    pass
        
        #충돌 위험이 없다면 다시 원래 max speed로 가속
        self.acc = (self.edge.max_speed - self.speed) / time_step
        
class AMHS:
    def __init__(self, nodes, edges, ports, num_OHTs, max_jobs, job_list = [], oht_list = []):
        """
        AMHS 초기화.
        - nodes: node 클래스 객체 리스트
        - edges: edge 클래스 객체 리스트
        - num_OHTs: 초기 OHT 수
        - max_jobs: 작업 큐의 최대 크기
        """
        # self.graph = nx.DiGraph()
        # self.original_graph=nx.DiGraph()
        
        self.graph = nk.Graph(directed=True, weighted=True)  # Directed weighted graph
        
        self.nodes = nodes  # node 객체 리스트
        for node in nodes:
            node.OHT = None
        self.edges = edges  # edge 객체 리스트
        for edge in edges:
            edge.OHTs = []
            edge.count = 0
            edge.avg_speed = edge.max_speed
        self.ports = ports #port list
        for p in ports:
            p.edge = self.get_edge(p.from_node, p.to_node)
        self.OHTs = []
        if job_list != []:
            self.job_queue = [[self.get_port(q[0]), self.get_port(q[1])] for q in job_list]
        else:
            self.job_queue = []
        self.max_jobs = max_jobs
        
        self.node_id_map = {}
        
        self.time_step=0.01
        
                
        self.simulation_running = False  # 시뮬레이션 상태 관리
        self.stop_simulation_event = threading.Event()  # 스레드 이벤트 처리

        # 그래프 생성
        self._create_graph()
        
        self.original_graph = nk.Graph(self.graph)

        # 초기 OHT 배치
        if oht_list != []:
            self.set_initial_OHTs(oht_list)
        else:
            self.initialize_OHTs(num_OHTs)
        
        self.apsp = nk.distance.APSP(self.graph)
        self.apsp.run()
        
        self.original_apsp = nk.distance.APSP(self.original_graph)
        self.original_apsp.run()
        


    def _create_graph(self):
        """NetworkX 그래프를 nodes와 edges로 생성."""
        for i, node in enumerate(self.nodes):
            self.graph.addNode()  # Add node to the graph
            self.node_id_map[node.id] = i
        
        for edge in self.edges:
            u = self.node_id_map[edge.source.id]  # Get index for source node
            v = self.node_id_map[edge.dest.id]    # Get index for destination node
            self.graph.addEdge(u, v, edge.length)
            edge.dest.incoming_edges.append(edge) #각 node마다 incoming edge 추가
            edge.source.outgoing_edges.append(edge)

    def initialize_OHTs(self, num_OHTs):
        available_nodes = self.nodes.copy()
        
        for i in range(num_OHTs):
            # 노드 중에서 랜덤 선택 (노드 소진 시 다시 전체에서 랜덤)
            if not available_nodes:
                available_nodes = self.nodes.copy()  # 노드 리스트 재생성
            
            start_node = random.choice(available_nodes)
            available_nodes.remove(start_node)  # 선택한 노드는 제거
            
            oht = OHT(
                id=i,
                from_node=start_node,
                from_dist=0,
                speed=0,
                acc=0,
                rect=1000,  # 충돌 판정 거리
                path=[],
            )
            start_node.OHT = oht
            self.OHTs.append(oht)
            
    def set_initial_OHTs(self, oht_list):
        i = 0
        for start_node in oht_list:  
            start_node = self.get_node(start_node)
            oht = OHT(
                id=i,
                from_node=start_node,
                from_dist=0,
                speed=0,
                acc=0,
                rect=1000,  # 충돌 판정 거리
                path=[],
            )
            start_node.OHT = oht
            self.OHTs.append(oht)
            i = i+1
    
    def set_oht(self, oht_origin, oht_new):
        oht_origin.from_node = self.get_node(oht_new['from_node'])
        oht_origin.from_dist = oht_new['from_dist']
        oht_origin.edge = self.get_edge(oht_new['source'], oht_new['dest'])
        if oht_origin.edge:
            if oht_origin not in oht_origin.edge.OHTs:
                oht_origin.edge.OHTs.append(oht_origin)
        try:
            if not self.graph.hasEdge(self.node_id_map[oht_new['source']], self.node_id_map[oht_new['dest']]):
                oht_origin.speed = 0
                oht_origin.acc = 0
                oht_origin.status = 'ON_REMOVED'
                oht_origin.cal_pos(self.time_step)
                return
        except:
            print('is here?', oht_new['source'])
  
            
        oht_origin.speed = oht_new['speed']
        oht_origin.status = oht_new['status']
        
        if oht_origin.status == 'ON_REMOVED':
            if oht_origin.start_port:
                oht_origin.status = "TO_START"
            elif oht_origin.end_port:
                oht_origin.status = "TO_END"
            else:
                oht_origin.status = "IDLE"
                
        oht_origin.cal_pos(self.time_step)
             
        
        oht_origin.start_port = self.get_port(oht_new['startPort']) if oht_new['startPort'] else None
        oht_origin.end_port = self.get_port(oht_new['endPort']) if oht_new['endPort'] else None
        oht_origin.wait_time = oht_new['wait_time']
                
        if oht_origin.start_port != None:
            if oht_origin.edge:
                path_edges_to_start = self.get_path_edges(oht_origin.edge.dest.id, oht_origin.start_port.to_node.id)
                path_edges_to_start = self._validate_path(path_edges_to_start)

            else:
                path_edges_to_start = self.get_path_edges(oht_origin.from_node.id, oht_origin.start_port.to_node.id)
                path_edges_to_start = self._validate_path(path_edges_to_start)
            oht_origin.path_to_start = path_edges_to_start[:]
        
        # elif oht_origin.status == "TO_END":
        if oht_origin.end_port != None:
            if oht_origin.status == "TO_END" or oht_origin.status == 'STOP_AT_START':
                if oht_origin.edge:
                    path_edges_to_end = self.get_path_edges(oht_origin.edge.dest.id, oht_origin.end_port.to_node.id)
                    path_edges_to_end = self._validate_path(path_edges_to_end)
                else:
                    path_edges_to_end = self.get_path_edges(oht_origin.from_node.id, oht_origin.end_port.to_node.id)
                    path_edges_to_end = self._validate_path(path_edges_to_end)
            else:
                path_edges_to_end = self.get_path_edges(oht_origin.start_port.to_node.id, oht_origin.end_port.to_node.id)

            oht_origin.path_to_end = path_edges_to_end
        
        if oht_origin.status == "TO_START":
            oht_origin.path = path_edges_to_start[:]
        elif oht_origin.status == "TO_END" or oht_origin.status == "STOP_AT_START":
            oht_origin.path = path_edges_to_end[:]
        else:
            oht_origin.path = []

    def modi_edge(self, source, dest, oht_positions, is_removed):
        if is_removed:
            removed_edge = self.get_edge(source, dest)
            try:       
                self.graph.removeEdge(self.node_id_map[source], self.node_id_map[dest])
                self.apsp = nk.distance.APSP(self.graph)
                self.apsp.run()
            except:
                print(source, dest)
        else:
            edge_to_restore = next((e for e in self.edges if e.source.id == source and e.dest.id == dest), None)
            if edge_to_restore:
                self.graph.addEdge(self.node_id_map[source], self.node_id_map[dest], edge_to_restore.length)
                self.apsp = nk.distance.APSP(self.graph)
                self.apsp.run()
            else:
                print(f"Rail {source} -> {dest} not found in original edges.")
        
    
    def reinitialize_simul(self, oht_positions, edge_data):
        for edge in self.edges:
            edge.OHTs.clear()
            edge.entry_exit_records.clear()
        
        for oht in oht_positions:
            oht_in_amhs = self.get_oht(oht['id'])
            self.set_oht(oht_in_amhs, oht)
            
        edge_map = {f"{edge.source.id}-{edge.dest.id}": edge for edge in self.edges}
    
        for edge_info in edge_data:
            edge_key = f"{edge_info['from']}-{edge_info['to']}"
            if edge_key in edge_map:
                edge = edge_map[edge_key]
                edge.count = edge_info.get('count', 0)
                edge.avg_speed = edge_info.get('avg_speed', 0)
            
        for edge in self.edges:
            edge.OHTs.sort(key = lambda oht : -oht.from_dist)
    
    def generate_job(self):
        """모든 OHT가 Job을 갖도록 작업 생성."""
        for _ in range(50):
            start_port = random.choice(self.ports)
            end_port = random.choice(self.ports)
            while start_port == end_port:  # 시작/목적 포트가 같지 않도록 보장
                end_port = random.choice(self.ports)
            self.job_queue.append((start_port, end_port))
            

    def assign_jobs(self):
        """모든 OHT가 Job을 갖도록 작업 할당."""
        closest_oht = None
        closest_dist = 1e100
        for oht in self.OHTs:
            if not oht.path and self.job_queue and oht.status == 'IDLE':  # OHT가 Idle 상태이고 Job Queue가 비어있지 않은 경우
                start_port, end_port = self.job_queue.pop(0)
                dist = self.get_path_distance(oht.from_node.id, start_port.from_node.id)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_oht = oht
        
        if closest_oht:
            oht = closest_oht
            oht.start_port = start_port
            # oht.start_port = next((port for port in self.ports if port.name == 'DIE0137_Port3'), None)
            oht.end_port = end_port
            oht.status = "TO_START"

            # # Start로 이동하는 경로
            if oht.edge:
                path_edges_to_start = self.get_path_edges(oht.edge.dest.id, start_port.to_node.id)
            else:
                path_edges_to_start = self.get_path_edges(oht.from_node.id, start_port.to_node.id)
            oht.path_to_start = path_edges_to_start[:]
            
            # # End로 이동하는 경로
            # start_edge = [self.get_edge(start_port.from_node.id, start_port.to_node.id)]
            path_edges_to_end = self.get_path_edges(start_port.to_node.id, end_port.to_node.id)
            oht.path_to_end = path_edges_to_end

            # # 전체 경로를 OHT에 설정
            oht.path = path_edges_to_start[:]
            
            # print(oht.path)

            # Assign the first edge in the path to the OHT's edge
            if oht.path:
                if not oht.edge:
                    oht.edge = oht.path.pop(0)
                    oht.from_node.OHT = None
                    if oht not in oht.edge.OHTs:
                        oht.edge.OHTs.append(oht)
                        
    def update_edge_metrics(self, current_time, time_window):
        for edge in self.edges:
            edge.avg_speed = edge.calculate_avg_speed(time_window, current_time)


    def get_node(self, node_id):
        """node id로 node 객체 반환."""
        return next((node for node in self.nodes if node.id == node_id), None)

    def get_edge(self, source_id, dest_id):
        """source와 dest ID로 edge 객체 반환."""
        return next(
            (e for e in self.edges if e.source.id == source_id and e.dest.id == dest_id), 
            None
        )
        
    def get_path_distance(self, source_id, dest_id):
        try:
            source_idx = self.node_id_map[source_id]
            dest_idx = self.node_id_map[dest_id]

            dist = self.apsp.getDistance(source_idx, dest_idx)      
            return dist
        except:
            try:
                source_idx = self.node_id_map[source_id]
                dest_idx = self.node_id_map[dest_id]            

                dist = self.original_apsp.getDistance(source_idx, dest_idx)                
                return dist
            except:
                print(f"No path found even in original_graph for {source_id} -> {dest_id}.")
                return float('inf')
        

    def get_path_edges(self, source_id, dest_id):
        """source와 dest ID로 최단 경로 에지 리스트 반환."""
        try:
            source_idx = self.node_id_map[source_id]
            dest_idx = self.node_id_map[dest_id]
            
            dijkstra = nk.distance.Dijkstra(self.graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
            dijkstra.run()

            path = dijkstra.getPath(dest_idx)
            
            return [
                self.get_edge(self.nodes[path[i]].id, self.nodes[path[i+1]].id)
                for i in range(len(path) - 1)
            ]
        except:
            try:
                source_idx = self.node_id_map[source_id]
                dest_idx = self.node_id_map[dest_id]
                
                dijkstra = nk.distance.Dijkstra(self.original_graph, source_idx, storePaths=True, storeNodesSortedByDistance=False, target=dest_idx)
                dijkstra.run()

                path = dijkstra.getPath(dest_idx)
                
                return [
                    self.get_edge(self.nodes[path[i]].id, self.nodes[path[i+1]].id)
                    for i in range(len(path) - 1)
                ]
            except:
                print(f"No path found even in original_graph for {source_id} -> {dest_id}.")
                return []
    
    def get_oht(self, oht_id):
        return next((oht for oht in self.OHTs if oht.id == oht_id), None)
    
    def get_port(self, port_name):
        return next((port for port in self.ports if port.name == port_name), None)

    def _validate_path(self, path):
        """
        현재 경로에 지워진 edge가 포함된 경우, 해당 edge 직전까지만 반환합니다.

        Parameters:
            path (list): 현재 경로 (edge 객체 리스트)

        Returns:
            list: 수정된 경로 (지워진 edge 직전까지만 포함)
        """
        valid_path = []
        for edge in path:
            if self.graph.hasEdge(self.node_id_map[edge.source.id], self.node_id_map[edge.dest.id]):  # edge가 그래프에 존재하는 경우
                valid_path.append(edge)
            else:
                break  # 지워진 edge를 발견하면 직전까지만 반환
        return valid_path
    
    
        
    def start_simulation(self, socketio, current_time, max_time = 4000, time_step = 0.1):
        """시뮬레이션 시작"""        
        if self.simulation_running:
            print("Simulation is already running. Stopping the current simulation...")
            self.stop_simulation_event.set()
            while self.simulation_running:
                socketio.sleep(0.01)  # Wait for the current simulation to stop
            return
        
        self.simulation_running = True
        self.stop_simulation_event.clear()
        
        count = 0
        edge_metrics_cache = {}  # Cache for edge metrics to track changes

        while current_time < max_time:
            if self.stop_simulation_event.is_set():
                break
            
            if count % 5 == 0:
                self.generate_job()
                self.assign_jobs()

            # Move all OHTs
            oht_positions = []
            for oht in self.OHTs:
                oht.move(time_step, current_time)

            self.update_edge_metrics(current_time, time_window=500)
            
            for oht in self.OHTs:
                oht.cal_pos(time_step)

            if count % 1 == 0:
                for oht in self.OHTs:
                    oht_positions.append({
                        'id': oht.id,
                        'x': oht.pos[0],
                        'y': oht.pos[1],
                        'source': oht.edge.source.id if oht.edge else None,
                        'dest': oht.edge.dest.id if oht.edge else None,
                        'speed': oht.speed,
                        'status': oht.status,
                        'startPort': oht.start_port.name if oht.start_port else None,
                        'endPort': oht.end_port.name if oht.end_port else None,
                        'from_node': oht.from_node.id if oht.from_node else None,
                        'from_dist': oht.from_dist,
                        'wait_time': oht.wait_time
                    })

                updated_edges = []
                for edge in self.edges:
                    key = f"{edge.source.id}-{edge.dest.id}"
                    new_metrics = {"count": edge.count, "avg_speed": edge.avg_speed}
                    if edge_metrics_cache.get(key) != new_metrics:
                        edge_metrics_cache[key] = new_metrics
                        updated_edges.append({
                            "from": edge.source.id,
                            "to": edge.dest.id,
                            **new_metrics
                        })

                payload = {
                    'time': current_time,
                    'oht_positions': oht_positions,
                    'edges': updated_edges
                }

                compressed_payload = compress_data(payload)
                socketio.emit('updateOHT', {'data': compressed_payload})

            # Increment time
            current_time += time_step
            count += 1
            socketio.sleep(0.00001)

        self.simulation_running = False
        print('Simulation ended')
        
        
    
    def accelerate_simul(self, socketio, current_time, max_time = 4000, time_step = 0.01):
        """시뮬레이션 시작"""        
        if self.simulation_running:
            print("Simulation is already running. Stopping the current simulation...")
            self.stop_simulation_event.set()
            while self.simulation_running:
                socketio.sleep(0.01)  # Wait for the current simulation to stop
            return
        
        self.simulation_running = True
        self.stop_simulation_event.clear()
        
        _current_time = 0
        
        count = 0

        while _current_time < current_time:
            if self.stop_simulation_event.is_set():
                break
            
            if count % 5 == 0:
                self.generate_job()
                self.assign_jobs()

            for oht in self.OHTs:
                oht.move(time_step, _current_time)

            self.update_edge_metrics(_current_time, time_window=500)
            
            for oht in self.OHTs:
                oht.cal_pos(time_step)

            # Increment time
            _current_time += time_step
            count += 1
            
        
        edge_metrics_cache = {} 
            
        
        while _current_time < max_time:
            if self.stop_simulation_event.is_set():
                break
            
            if count % 5 == 0:
                self.generate_job()
                self.assign_jobs()

            # Move all OHTs
            oht_positions = []
            for oht in self.OHTs:
                oht.move(time_step, _current_time)

            self.update_edge_metrics(_current_time, time_window=500)
            
            for oht in self.OHTs:
                oht.cal_pos(time_step)

            if count % 1 == 0:
                for oht in self.OHTs:
                    oht_positions.append({
                        'id': oht.id,
                        'x': oht.pos[0],
                        'y': oht.pos[1],
                        'source': oht.edge.source.id if oht.edge else None,
                        'dest': oht.edge.dest.id if oht.edge else None,
                        'speed': oht.speed,
                        'status': oht.status,
                        'startPort': oht.start_port.name if oht.start_port else None,
                        'endPort': oht.end_port.name if oht.end_port else None,
                        'from_node': oht.from_node.id if oht.from_node else None,
                        'from_dist': oht.from_dist,
                        'wait_time': oht.wait_time
                    })

                updated_edges = []
                for edge in self.edges:
                    key = f"{edge.source.id}-{edge.dest.id}"
                    new_metrics = {"count": edge.count, "avg_speed": edge.avg_speed}
                    if edge_metrics_cache.get(key) != new_metrics:
                        edge_metrics_cache[key] = new_metrics
                        updated_edges.append({
                            "from": edge.source.id,
                            "to": edge.dest.id,
                            **new_metrics
                        })

                payload = {
                    'time': current_time,
                    'oht_positions': oht_positions,
                    'edges': updated_edges
                }

                compressed_payload = compress_data(payload)
                socketio.emit('updateOHT', {'data': compressed_payload})

            # Increment time
            current_time += time_step
            count += 1
            socketio.sleep(0.00001)

        self.simulation_running = False
        print('Simulation ended')